# SpectraMind V50 — Master Walkthrough (One Notebook)

End‑to‑end **single notebook** that mirrors the CLI‑first SpectraMind V50 pipeline and bundles the physics demos.

**Contents**
1. Environment & CLI check
2. Quickstart smoke (optional)
3. Calibration (pipeline‑safe)
4. Train (fast path)
5. Predict / Submit bundle (dry‑run)
6. Diagnostics (summary + HTML/PNG artifacts)
7. Uncertainty calibration sketch (educational)
8. Radiation & Noise Modeling (educational)
9. Gravitational Lensing Demo (educational)
10. Appendix — CLI cheat‑sheet


In [ ]:
import os, sys, json, shutil, subprocess, platform, textwrap
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

ROOT = Path.cwd().resolve()
NB_OUT = ROOT / 'outputs' / 'notebooks' / 'master_walkthrough'
NB_OUT.mkdir(parents=True, exist_ok=True)

ENV = {
    'python': platform.python_version(),
    'platform': platform.platform(),
    'time': datetime.utcnow().isoformat()+'Z',
}
(NB_OUT/'env_snapshot.json').write_text(json.dumps(ENV, indent=2))

def have(cmd):
    return shutil.which(cmd) is not None

CLI = 'spectramind'
print('ROOT:', ROOT)
print('NB_OUT:', NB_OUT)
print('CLI present?', have(CLI))

## 1) Environment & CLI check
This notebook calls the unified CLI when available. If the `spectramind` CLI is not on PATH, cells will **dry‑run** with safe fallbacks so the notebook remains self‑contained.

## 2) Quickstart smoke (optional)
Runs a tiny smoke test to ensure configs & paths resolve.

In [ ]:
if have(CLI):
    try:
        print('> spectramind test --fast')
        _ = subprocess.run([CLI, 'test', '--fast'], check=False)
    except Exception as e:
        print('Smoke skipped:', e)
else:
    print('CLI not found: quickstart smoke skipped (ok).')

## 3) Calibration (pipeline‑safe)
Calibrate raw frames to science‑ready light‑curves. Uses a minimal subset for speed if supported.

In [ ]:
if have(CLI):
    try:
        print('> spectramind calibrate --sample 5')
        _ = subprocess.run([CLI, 'calibrate', '--sample', '5'], check=False)
    except Exception as e:
        print('Calibration skipped:', e)
else:
    print('CLI not found: emitting placeholder calibrated artifact…')
    # Create a tiny placeholder CSV representing a calibrated spectrum
    wl = np.arange(283)
    mu = 0.01 + 0.002*np.exp(-0.5*((wl/283-0.35)/0.08)**2)
    pd.DataFrame({'wavelength_index': wl, 'mu': mu}).to_csv(NB_OUT/'calibrated_placeholder.csv', index=False)
    print('Wrote', (NB_OUT/'calibrated_placeholder.csv').as_posix())

## 4) Train (fast path)
Fast‑dev run to validate shapes, configs, and logging.

In [ ]:
if have(CLI):
    try:
        print('> spectramind train --epochs 1 --fast-dev')
        _ = subprocess.run([CLI, 'train', '--epochs', '1', '--fast-dev'], check=False)
    except Exception as e:
        print('Train skipped:', e)
else:
    print('CLI not found: training step skipped (ok).')

## 5) Predict / Submit bundle (dry‑run)
Bundle predictions and submission manifest (dry‑run when CLI is absent).

In [ ]:
if have(CLI):
    try:
        print('> spectramind submit --dry-run')
        _ = subprocess.run([CLI, 'submit', '--dry-run'], check=False)
    except Exception as e:
        print('Submit skipped:', e)
else:
    print('CLI not found: creating a placeholder submission.csv…')
    # Minimal placeholder
    df = pd.DataFrame({'planet_id':[0], **{f'mu_{i}':[0.0] for i in range(283)}})
    df.to_csv(NB_OUT/'submission_placeholder.csv', index=False)
    print('Wrote', (NB_OUT/'submission_placeholder.csv').as_posix())

## 6) Diagnostics (summary)
If the CLI saved HTML/PNG reports in outputs, link them here; otherwise produce a tiny overview plot.

In [ ]:
diag_png = NB_OUT/'quick_diag.png'
wl = np.arange(283)
mu = 0.01 + 0.002*np.exp(-0.5*((wl/283-0.30)/0.06)**2) + 0.0015*np.exp(-0.5*((wl/283-0.70)/0.05)**2)
plt.figure(figsize=(10,3))
plt.plot(wl, mu, lw=1.5)
plt.title('Quick diagnostic spectrum (placeholder)')
plt.xlabel('wavelength index'); plt.ylabel('μ (arb)')
plt.tight_layout(); plt.savefig(diag_png, dpi=150); plt.close()
print('Saved', diag_png)

## 7) Uncertainty calibration (educational sketch)
Plot a synthetic **σ vs residual** calibration check to illustrate the idea (not pipeline data).

In [ ]:
np.random.seed(0)
n = 1000
pred_sigma = np.clip(np.random.lognormal(mean=-5.0, sigma=0.5, size=n), 1e-6, 5e-2)
residual = pred_sigma * np.random.normal(0,1,size=n)
plt.figure(figsize=(5,4))
plt.scatter(pred_sigma, np.abs(residual), s=6, alpha=0.4)
plt.plot([pred_sigma.min(), pred_sigma.max()], [pred_sigma.min(), pred_sigma.max()], lw=2)
plt.xscale('log'); plt.yscale('log')
plt.xlabel('predicted σ'); plt.ylabel('|residual|')
plt.title('Calibration sketch (ideal: points along y=x)')
plt.tight_layout(); plt.savefig(NB_OUT/'uncertainty_calibration_sketch.png', dpi=150); plt.close()
print('Saved uncertainty_calibration_sketch.png')

## 8) Radiation & Noise Modeling (educational)
Demonstrate Poisson shot noise, Gaussian read noise, 1/f noise, and cosmic ray spikes on a base spectrum; then compare FFT/autocorr.

In [ ]:
rng = np.random.default_rng(1234)
x = np.linspace(0,1,283)
mu_base = 0.01 + 0.002*np.exp(-0.5*((x-0.35)/0.08)**2) + 0.0015*np.exp(-0.5*((x-0.75)/0.05)**2)

def add_shot(mu, scale_counts=3e5):
    lam = np.clip(scale_counts*np.maximum(mu,0), 0, None)
    return rng.poisson(lam)/scale_counts
def add_read(mu, sigma=2e-4):
    return mu + rng.normal(0.0, sigma, size=mu.shape)
def add_1f(mu, alpha=1.0, amp=2e-4):
    n=len(mu); white=rng.normal(0,1,n)
    f=np.fft.rfftfreq(n); spec=np.fft.rfft(white)
    shaping=1.0/np.maximum(f,1e-6)**alpha
    shaped=spec*shaping
    x_ifft=np.fft.irfft(shaped, n)
    x_ifft=amp*x_ifft/np.std(x_ifft)
    return mu + x_ifft
def add_cr(mu, n_hits=3, amp=0.01, kernel=(1.0,0.5)):
    y=mu.copy(); W=len(mu); hits=rng.choice(W, size=min(n_hits,W), replace=False)
    for h in hits:
        for k,a in enumerate(kernel):
            idx=h+k
            if idx<W:
                y[idx]+=a*amp
    return y, hits

noisy_shot   = add_shot(mu_base)
noisy_read   = add_read(mu_base)
noisy_1f     = add_1f(mu_base)
noisy_cr, H  = add_cr(mu_base, n_hits=4, amp=0.01)

fig, ax = plt.subplots(2,2, figsize=(12,6), sharex=True)
ax=ax.ravel()
ax[0].plot(mu_base, lw=1.2, label='base'); ax[0].plot(noisy_shot, lw=0.9, label='shot'); ax[0].legend(); ax[0].set_title('Shot')
ax[1].plot(mu_base, lw=1.2, label='base'); ax[1].plot(noisy_read, lw=0.9, label='read'); ax[1].legend(); ax[1].set_title('Read')
ax[2].plot(mu_base, lw=1.2, label='base'); ax[2].plot(noisy_1f, lw=0.9, label='1/f'); ax[2].legend(); ax[2].set_title('1/f')
ax[3].plot(mu_base, lw=1.2, label='base'); ax[3].plot(noisy_cr, lw=0.9, label='cosmic'); ax[3].scatter(H, noisy_cr[H], s=22)
ax[3].legend(); ax[3].set_title('Cosmic rays')
for a in ax: a.set_ylabel('μ (arb)')
ax[2].set_xlabel('wavelength index'); ax[3].set_xlabel('wavelength index')
fig.tight_layout(); fig.savefig(NB_OUT/'noise_panels.png', dpi=150); plt.close(fig)
print('Saved noise_panels.png')

def fft_power(y):
    y=y-np.nanmean(y)
    fy=np.fft.rfft(y); p=np.abs(fy)**2; f=np.fft.rfftfreq(len(y))
    return f, p
def acorr(y):
    y=y-np.nanmean(y)
    r=np.correlate(y,y,mode='full'); r=r[r.size//2:]
    if r[0]!=0: r=r/r[0]
    l=np.arange(r.size); return l,r

series={'base':mu_base,'shot':noisy_shot,'read':noisy_read,'1f':noisy_1f,'cr':noisy_cr}
fig, ax = plt.subplots(2,2, figsize=(12,6)); ax=ax.ravel()
for i,(name, y) in enumerate(series.items()):
    f,p=fft_power(y)
    ax[0].semilogy(f[1:], p[1:], lw=1, label=name)
ax[0].legend(); ax[0].set_title('FFT power (log)'); ax[0].set_xlabel('freq'); ax[0].set_ylabel('power')
for i,(name, y) in enumerate(series.items()):
    l,r = acorr(y)
    ax[1].plot(l, r, lw=1, label=name)
ax[1].legend(); ax[1].set_title('Autocorrelation'); ax[1].set_xlabel('lag'); ax[1].set_ylabel('norm acorr')
fig.tight_layout(); fig.savefig(NB_OUT/'fft_autocorr_compare.png', dpi=150); plt.close(fig)
print('Saved fft_autocorr_compare.png')

## 9) Gravitational Lensing Demo (educational)
Show how time‑varying microlensing **A(t)** during a transit plus wavelength‑dependent weights can bias an effective spectrum.

In [ ]:
def A_point_lens(u):
    u = np.asarray(u, float)
    return (u*u + 2) / (u * np.sqrt(u*u + 4))
def u_of_t(t, t0=0.0, u0=0.2, tE=0.25):
    return np.sqrt(u0*u0 + ((t - t0)/tE)**2)

N_EXP=200
t = np.linspace(-0.8, 0.8, N_EXP)
A = A_point_lens(u_of_t(t, t0=0.0, u0=0.20, tE=0.25))
plt.figure(figsize=(8,3)); plt.plot(t, A, lw=1.6)
plt.title('Microlensing magnification A(t)'); plt.xlabel('time [arb]'); plt.ylabel('A')
plt.tight_layout(); plt.savefig(NB_OUT/'magnification_curve.png', dpi=150); plt.close()
print('Saved magnification_curve.png')

wl = np.arange(283)
mu = 0.01 + 0.002*np.exp(-0.5*((wl/283-0.30)/0.06)**2) + 0.0015*np.exp(-0.5*((wl/283-0.70)/0.05)**2)
W = wl / wl.max(); W = 0.5 + 0.5*(W - W.min())/(W.ptp()+1e-12)
phase_per_wl = np.linspace(-0.15, 0.15, len(wl))
A_per_wl = np.interp(phase_per_wl, t, A)
mu_lensed = mu * (1.0 + (A_per_wl - 1.0)*W)
res = mu_lensed - mu
fig, ax = plt.subplots(2,1, figsize=(10,6), sharex=True)
ax[0].plot(wl, mu, lw=1.4, label='base μ'); ax[0].plot(wl, mu_lensed, lw=1.2, label='effective μ'); ax[0].legend(); ax[0].set_ylabel('μ')
ax[1].plot(wl, res, lw=1.2, color='tab:red'); ax[1].axhline(0, color='k', lw=0.7)
ax[1].set_xlabel('wavelength index'); ax[1].set_ylabel('Δμ (arb)')
fig.suptitle('Effective spectrum under microlensing + wavelength weights')
fig.tight_layout(); fig.savefig(NB_OUT/'effective_spectrum_lensing.png', dpi=150); plt.close(fig)
print('Saved effective_spectrum_lensing.png')

## 10) Appendix — CLI cheat‑sheet
```
spectramind test --fast
spectramind calibrate --sample 5
spectramind train --epochs 1 --fast-dev
spectramind submit --dry-run
spectramind diagnose --open-html
```

Artifacts written to `outputs/notebooks/master_walkthrough/`.

---
# 🔗 Stitched Modules (auto-imported)
The following sections were imported from the uploaded notebooks to keep a single master file.

## 00_quickstart.ipynb

# 🚀 SpectraMind V50 — Quickstart Notebook

Fast path to validate your **environment → configs → pipeline** for the **NeurIPS 2025 Ariel Data Challenge**.

This notebook mirrors our **CLI-first, Hydra-driven, reproducibility-focused** workflow (Typer CLI + Hydra configs + DVC) so you can smoke-test the stack in minutes without re-implementing any logic.

⚠️ **Note:** All steps default to `--fast`, `--dry-run`, or sample subsets. Switch to full runs only after these checks pass.


## 0. Runtime Helper — Resolve `spectramind` Launcher

Some environments expose the CLI differently. This helper resolves in order:
1. `spectramind` (on PATH)
2. `poetry run spectramind`
3. `python -m spectramind`

Provides `sm(cmd)` and `sm_print(cmd)` helpers.

In [ ]:
import os, shlex, shutil, subprocess, sys

def _resolve_spectramind_cmd():
    if shutil.which("spectramind"):
        return ["spectramind"]
    if shutil.which("poetry"):
        try:
            out = subprocess.run(["poetry", "run", "spectramind", "--version"],
                                 stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
            if out.returncode == 0:
                return ["poetry", "run", "spectramind"]
        except Exception:
            pass
    return [sys.executable, "-m", "spectramind"]

SM = _resolve_spectramind_cmd()
print("Resolved spectramind launcher:", " ".join(shlex.quote(p) for p in SM))

def sm(cmd: str, check=False, capture=False):
    args = shlex.split(cmd)
    res = subprocess.run(SM + args,
                         check=check,
                         stdout=subprocess.PIPE if capture else None,
                         stderr=subprocess.STDOUT if capture else None,
                         text=True)
    return res

def sm_print(cmd: str):
    print("$", " ".join(SM + shlex.split(cmd)))


## 1. Repo Sanity Checks
Validate core tooling (Python, Poetry, Git, DVC) and GPU snapshot.

In [ ]:
!python --version
!pip --version
!poetry --version || echo '⚠️ Poetry not found'
!git --version
!dvc --version || echo '⚠️ DVC not found'

try:
    import torch
    print("PyTorch:", torch.__version__)
    if torch.cuda.is_available():
        print("CUDA ✓ — device:", torch.cuda.get_device_name(0))
    else:
        print("CUDA not available (CPU)")
except Exception:
    print("PyTorch not installed (ok for quickstart)")


## 2. CLI Smoke Test

In [ ]:
sm_print("--version")
out = sm("--version", capture=True)
print(out.stdout)

sm_print("--help")
out = sm("--help", capture=True)
print("\n".join(out.stdout.splitlines()[:20]))


## 3. Hydra Config Composition

In [ ]:
cmd = "train model=airs_gnn optimizer=adamw training.fast_dev_run=true"
sm_print(cmd)
sm(cmd)


## 4. Mini End-to-End Pipeline

In [ ]:
for cmd in [
    "test --fast",
    "calibrate --sample 3 --fast",
    "train --epochs 1 training.fast_dev_run=true",
    "diagnose dashboard --no-umap --no-tsne --outdir outputs/diagnostics_quick",
    "submit --dry-run"
]:
    sm_print(cmd)
    sm(cmd)


## 5. Cheat Sheet — Common Workflows
```bash
spectramind train model=fgs1_mamba optimizer=adamw training.epochs=50
spectramind calibrate --sample 10
spectramind submit --config configs/config_v50.yaml
```
Logs land in `logs/v50_debug_log.md`.

## 6. DVC Data Tips

In [ ]:
%%bash
set -euo pipefail
if command -v dvc >/dev/null 2>&1; then
  echo "DVC detected — pulling latest data..."
  dvc pull || echo '⚠️ non-fatal'
  echo "Optionally recompute: dvc repro"
else
  echo "DVC not installed — skipping"
fi


## 7. Python Helper — Run CLI Inside Notebook

In [ ]:
res = sm("--version", capture=True)
print(res.stdout)

res = sm("train training.fast_dev_run=true", capture=True)
for line in (res.stdout or "").splitlines():
    if "loss" in line.lower() or "gll" in line.lower():
        print(line)


## 8. Where to Look Afterwards
- Outputs: `outputs/`
- Logs: `logs/v50_debug_log.md`
- Diagnostics HTML: from `diagnose dashboard`
- Configs: `configs/`


## 01_pipeline_calibrate_train_predict.ipynb

# 🧪 SpectraMind V50 — 01_pipeline_calibrate_train_predict

Tiny **calibrate → train → predict** pipeline to sanity-check the stack with fast settings.

This runs entirely via the **CLI** (Typer + Hydra), keeping the workflow reproducible and config-driven.

## 0) Runtime Helper — Resolve `spectramind`

Resolves in order:
1. `spectramind` (PATH)
2. `poetry run spectramind`
3. `python -m spectramind`

Exposes helpers: `sm(cmd)`, `sm_print(cmd)`.

In [ ]:
import os, shlex, shutil, subprocess, sys, glob, json, textwrap
from pathlib import Path

def _resolve_spectramind_cmd():
    if shutil.which("spectramind"):
        return ["spectramind"]
    if shutil.which("poetry"):
        try:
            out = subprocess.run(["poetry", "run", "spectramind", "--version"],
                                 stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
            if out.returncode == 0:
                return ["poetry", "run", "spectramind"]
        except Exception:
            pass
    return [sys.executable, "-m", "spectramind"]

SM = _resolve_spectramind_cmd()
print("Resolved spectramind launcher:", " ".join(shlex.quote(p) for p in SM))

def sm(cmd: str, check=False, capture=False):
    args = shlex.split(cmd)
    res = subprocess.run(SM + args,
                         check=check,
                         stdout=subprocess.PIPE if capture else None,
                         stderr=subprocess.STDOUT if capture else None,
                         text=True)
    return res

def sm_print(cmd: str):
    print("$", " ".join(SM + shlex.split(cmd)))

Path("outputs").mkdir(exist_ok=True, parents=True)
Path("outputs/preds_quick").mkdir(exist_ok=True, parents=True)
Path("outputs/diagnostics_quick").mkdir(exist_ok=True, parents=True)

## 1) Sanity Checks
Python/Poetry/Git/DVC and optional CUDA snapshot.

In [ ]:
!python --version
!pip --version
!poetry --version || echo '⚠️ Poetry not found (ok if not using Poetry)'
!git --version
!dvc --version || echo '⚠️ DVC not found (ok for quick run)'

try:
    import torch
    print("PyTorch:", torch.__version__)
    print("CUDA available:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("CUDA device:", torch.cuda.get_device_name(0))
except Exception:
    print("PyTorch not installed — continuing")

sm_print("--version")
out = sm("--version", capture=True)
print(out.stdout if out.stdout else "(no output)")

sm_print("--help")
out = sm("--help", capture=True)
print("\n".join(out.stdout.splitlines()[:20]) if out.stdout else "(no output)")

## 2) (Optional) DVC Pull
If your repo uses DVC for data/artifacts, pull latest (non-fatal if absent).

In [ ]:
%%bash
set -euo pipefail
if command -v dvc >/dev/null 2>&1; then
  echo "DVC detected — pulling data (if remote configured)..."
  dvc pull || echo '⚠️ dvc pull non-fatal'
else
  echo "DVC not installed — skipping"
fi

## 3) Calibrate (fast, sampled)
Run calibration on a small sample for speed. Adjust `--sample` as needed.

In [ ]:
sm_print("calibrate --sample 5 --fast")
cal = sm("calibrate --sample 5 --fast", capture=True)
print(cal.stdout or "(no output)")

# List key outputs if your pipeline writes calibrated artifacts
!ls -lah outputs || true
!ls -lah outputs/* || true 2>/dev/null || true
!find outputs -maxdepth 2 -type f | head -n 20 || true

## 4) Train (tiny)
One quick epoch / fast-dev-run for smoke validation. Override Hydra config inline if desired.

In [ ]:
train_cmd = "train --epochs 1 training.fast_dev_run=true"
sm_print(train_cmd)
tr = sm(train_cmd, capture=True)
print(tr.stdout or "(no output)")

# Peek at any saved checkpoints/metrics
!find outputs -maxdepth 3 -type f \( -name "*.pt" -o -name "*.ckpt" -o -name "metrics*.json" -o -name "*log*.json" \) | head -n 20 || true
!tail -n 50 logs/v50_debug_log.md 2>/dev/null || true
!tail -n 50 outputs/metrics.json 2>/dev/null || true
!tail -n 50 outputs/train/metrics.json 2>/dev/null || true
!find outputs -maxdepth 3 -type f -name "*.json" | head -n 10 || true
!find outputs -maxdepth 3 -type f -name "*.yaml" | head -n 10 || true

## 5) Predict
Generate quick predictions (location may vary by your CLI). This writes predictions to `outputs/preds_quick/`.

- If your CLI exposes `predict`, use that.
- If predictions are created by `submit --dry-run`, run that and parse the produced CSV/ZIP accordingly.

In [ ]:
pred_dir = Path("outputs/preds_quick")

# Try `predict` first
predict_cmds = [
    "predict --outdir outputs/preds_quick --fast",
    # Fallback: some repos produce predictions as part of submit dry-run
    "submit --dry-run"
]

ok = False
for cmd in predict_cmds:
    sm_print(cmd)
    out = sm(cmd, capture=True)
    print(out.stdout or "(no output)")
    # Heuristic: check if something landed in preds_quick or a submission file exists
    csvs = list(pred_dir.glob("**/*.csv"))
    subs = list(Path(".").glob("**/submission*.csv"))
    if csvs or subs:
        ok = True
        break

if not ok:
    print("⚠️ Could not detect predictions in expected locations. Inspect CLI outputs above.")

print("\nDiscovered prediction files:")
for p in pred_dir.glob("**/*.csv"):
    print("-", p)
for p in Path(".").glob("**/submission*.csv"):
    print("-", p)

!ls -lah outputs/preds_quick 2>/dev/null || true
!find outputs -maxdepth 2 -type f -name "*.csv" | head -n 20 || true
!find . -maxdepth 3 -type f -name "submission*.csv" | head -n 10 || true

## 6) Inspect Predictions (preview & quick plot)
This section attempts to open a CSV of predictions and visualize a spectrum for a quick sanity check (optional).

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def _find_pred_csv():
    # prefer explicit preds directory, else any submission*.csv
    cands = list(Path("outputs/preds_quick").glob("**/*.csv"))
    if cands:
        return cands[0]
    cands = list(Path(".").glob("**/submission*.csv"))
    return cands[0] if cands else None

csv_path = _find_pred_csv()
if csv_path and csv_path.is_file():
    print("Preview:", csv_path)
    df = pd.read_csv(csv_path)
    display(df.head(10))
    # Try naive plot: look for columns that look like wavelength bins
    num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
    # If a single row contains a full spectrum across columns, plot the first row
    if len(num_cols) > 10:
        y = df[num_cols].iloc[0].values
        x = list(range(len(y)))
        plt.figure(figsize=(8,3))
        plt.plot(x, y, lw=1)
        plt.title(f"Quick Spectrum Preview — {csv_path.name}")
        plt.xlabel("Bin index")
        plt.ylabel("Predicted μ")
        plt.tight_layout()
        plt.show()
    else:
        print("(Skipping plot — could not infer wide spectrum columns)")
else:
    print("No prediction CSV found to preview.")

## 7) (Optional) Diagnostics Snapshot
Run a minimal dashboard build to confirm plots render (UMAP/t-SNE disabled for speed). Outputs in `outputs/diagnostics_quick/`.

In [ ]:
diag_cmd = "diagnose dashboard --no-umap --no-tsne --outdir outputs/diagnostics_quick"
sm_print(diag_cmd)
dg = sm(diag_cmd, capture=True)
print(dg.stdout or "(no output)")

!find outputs/diagnostics_quick -maxdepth 2 -type f | head -n 20 || true
!ls -lah outputs/diagnostics_quick 2>/dev/null || true
!find outputs -maxdepth 3 -type f -name "*report*.html" | head -n 10 || true

## 8) Summary & Next Steps
- You now have calibrated data, a quick-trained model, and a prediction artifact.
- For real experiments, increase `--sample`, remove `--fast_dev_run`, and raise `--epochs`.
- Consider running `submit` without `--dry-run` to package a full submission when ready.
- Explore diagnostics HTML under `outputs/diagnostics_quick/`.

## 02_diagnostics_explainability.ipynb

# 🧭 SpectraMind V50 — 02 · Diagnostics & Explainability

**NeurIPS 2025 Ariel Data Challenge** · *NASA‑grade reproducibility w/ Hydra + DVC + CLI*

This notebook focuses on diagnostics and explainability artifacts produced by the SpectraMind V50 pipeline.
It is *CLI‑first*, meaning: wherever possible, we **call the `spectramind` CLI** to generate artifacts,
then **render them here**. Each section also includes a *graceful fallback* path that reads existing files
(e.g., `diagnostic_summary.json`, `*.html`, `*.csv`, `*.npy`) directly if present.

> Golden rule: **No hidden analytics**. The GUI/notebook **reflects** what the CLI generated to keep faithful,
> reproducible results. Cells here log their actions and record run metadata for later auditing.


## ✅ What you can do here

1. Run *lightweight* CLI diagnostics (UMAP/t‑SNE, SHAP overlays, symbolic rule analysis, FFT/smoothness, calibration).
2. Render previously generated HTML dashboards and plots.
3. Inspect metrics from `diagnostic_summary.json` across planets and configs.
4. Export a refreshed diagnostics dashboard and append metadata into `v50_debug_log.md`.

> Tip: Start with the **Environment & Paths** cell below, then try **Quick CLI sanity** to verify your setup.


## 🔧 Environment & Paths

In [ ]:
# This cell sets up common paths and logs environment context.
# Adjust ROOT to your repository root if needed.
from pathlib import Path
import json, os, sys, platform, shutil, textwrap
from datetime import datetime

# ---- Repository root deduction: prefer CWD; allow manual override ----
ROOT = Path.cwd()
ARTEFACTS = ROOT / "artifacts"
DIAG_DIR = ARTEFACTS / "diagnostics"
HTML_DIR = DIAG_DIR / "html"
PLOTS_DIR = DIAG_DIR / "plots"

# Common artifact inputs (created by CLI)
DIAG_SUMMARY = DIAG_DIR / "diagnostic_summary.json"
SHAP_JSON = DIAG_DIR / "shap_symbolic_fusion_topk_bins.json"
SYMB_JSON = DIAG_DIR / "symbolic_violation_summary.json"
COREL_JSON = DIAG_DIR / "corel_calibration_summary.json"
UMAP_HTML = HTML_DIR / "umap_v50.html"
TSNE_HTML = HTML_DIR / "tsne_interactive.html"
DASHBOARD_HTML = HTML_DIR / "diagnostic_report_v1.html"
LOG_MD = ROOT / "v50_debug_log.md"

# Optional latent/label fallbacks
LATENTS_NPY = DIAG_DIR / "latents.npy"
LATENTS_CSV = DIAG_DIR / "latents.csv"
LABELS_CSV = DIAG_DIR / "labels.csv"

# Create folders if missing (no-op if existing)
for d in [ARTEFACTS, DIAG_DIR, HTML_DIR, PLOTS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

env = {
    "timestamp": datetime.now().isoformat(timespec="seconds"),
    "python": sys.version.replace("\n", " "),
    "platform": platform.platform(),
    "cwd": str(ROOT),
    "paths": {
        "ARTEFACTS": str(ARTEFACTS),
        "DIAG_DIR": str(DIAG_DIR),
        "HTML_DIR": str(HTML_DIR),
        "PLOTS_DIR": str(PLOTS_DIR),
        "DIAG_SUMMARY": str(DIAG_SUMMARY),
        "SHAP_JSON": str(SHAP_JSON),
        "SYMB_JSON": str(SYMB_JSON),
        "COREL_JSON": str(COREL_JSON),
        "UMAP_HTML": str(UMAP_HTML),
        "TSNE_HTML": str(TSNE_HTML),
        "DASHBOARD_HTML": str(DASHBOARD_HTML),
        "LOG_MD": str(LOG_MD),
        "LATENTS_NPY": str(LATENTS_NPY),
        "LATENTS_CSV": str(LATENTS_CSV),
        "LABELS_CSV": str(LABELS_CSV),
    },
}
print(json.dumps(env, indent=2))


## 🩺 Quick CLI sanity (optional)

In [ ]:
# This cell *optionally* checks CLI availability. It's safe to skip if your CLI isn't on PATH.
# In many notebook runtimes, subprocess may not find your local CLI; that's okay.
import shutil, subprocess

def check_cli(cmd="spectramind", args=["--version"]):
    exe = shutil.which(cmd)
    if not exe:
        print("⚠️ 'spectramind' CLI not found on PATH. Skipping CLI sanity check.")
        return {"available": False}
    try:
        out = subprocess.check_output([cmd] + args, stderr=subprocess.STDOUT, text=True, timeout=30)
        print(out)
        return {"available": True, "output": out}
    except Exception as e:
        print(f"⚠️ CLI call failed: {e}")
        return {"available": True, "error": str(e)}

cli_info = check_cli()
cli_info


## 🌈 UMAP Diagnostics — generate or load

In [ ]:
# Attempt to call the CLI to generate UMAP HTML; otherwise load existing HTML if present.
import subprocess, webbrowser

def run_umap_cli():
    try:
        exe = shutil.which("spectramind")
        if not exe:
            return False, "CLI not found"
        # Minimal example — adjust flags to your configs
        cmd = ["spectramind", "diagnose", "umap",
               "--html-out", str(UMAP_HTML),
               "--log-file", str(LOG_MD),
               "--open-browser", "false"]
        print("Running:", " ".join(cmd))
        subprocess.check_call(cmd, timeout=600)
        return True, "OK"
    except Exception as e:
        return False, str(e)

ok, msg = run_umap_cli()
if ok:
    print(f"✅ UMAP generated at: {UMAP_HTML}")
elif UMAP_HTML.exists():
    print("⚠️ CLI skipped/failed; using existing:", UMAP_HTML)
else:
    print("❌ UMAP not available; create latents or run CLI separately.")

# Inline display hint (cannot iframe automatically in all notebook environments)
print("To open UMAP HTML manually if needed:", str(UMAP_HTML))


## 🌀 t‑SNE Diagnostics — generate or load

In [ ]:
# Attempt to call CLI t‑SNE; otherwise load existing HTML if present.
def run_tsne_cli():
    try:
        exe = shutil.which("spectramind")
        if not exe:
            return False, "CLI not found"
        cmd = ["spectramind", "diagnose", "tsne-latents",
               "--html-out", str(TSNE_HTML),
               "--log-file", str(LOG_MD),
               "--open-browser", "false"]
        print("Running:", " ".join(cmd))
        subprocess.check_call(cmd, timeout=600)
        return True, "OK"
    except Exception as e:
        return False, str(e)

ok, msg = run_tsne_cli()
if ok:
    print(f"✅ t‑SNE generated at: {TSNE_HTML}")
elif TSNE_HTML.exists():
    print("⚠️ CLI skipped/failed; using existing:", TSNE_HTML)
else:
    print("❌ t‑SNE not available; ensure latents exist or run CLI separately.")

print("To open t‑SNE HTML manually if needed:", str(TSNE_HTML))


### 🔁 Fallback: quickscatter for latents (matplotlib)

In [ ]:
# If HTMLs are not present, try to render a simple 2D scatter from CSV/NPY latents.
# Matplotlib only (no seaborn, and a single plot per chart per project constraints).
import numpy as np
import matplotlib.pyplot as plt

def load_matrix(path):
    if path.suffix == ".csv":
        import pandas as pd
        return pd.read_csv(path, index_col=None).values
    elif path.suffix == ".npy":
        return np.load(path)
    else:
        raise ValueError(f"Unsupported format for {path}")

latent_matrix = None
for p in [LATENTS_CSV, LATENTS_NPY]:
    if p.exists():
        try:
            latent_matrix = load_matrix(p)
            print(f"Loaded latents from {p} with shape {latent_matrix.shape}")
            break
        except Exception as e:
            print(f"Failed to load {p}: {e}")

if latent_matrix is not None:
    # Use first 2 columns as a crude projection
    x = latent_matrix[:, 0]
    y = latent_matrix[:, 1] if latent_matrix.shape[1] > 1 else np.zeros_like(x)
    plt.figure(figsize=(6, 5))
    plt.scatter(x, y, s=10, alpha=0.7)
    plt.title("Latent Quickscatter (fallback)")
    plt.xlabel("Dim 1")
    plt.ylabel("Dim 2")
    plt.show()
else:
    print("No latents found for fallback quickscatter.")


## 🔍 SHAP × Symbolic Overlay Inspection

In [ ]:
# Load SHAP/symbolic overlay JSON if present; show top-K bins per planet or aggregate stats.
import json
from collections import Counter, defaultdict

def safe_load_json(path):
    if not path.exists():
        return None
    try:
        with open(path, "r", encoding="utf-8") as f:
            return json.load(f)
    except Exception as e:
        print(f"Failed to parse {path}: {e}")
        return None

overlay = safe_load_json(SHAP_JSON)
if overlay is None:
    print(f"⚠️ No overlay JSON at {SHAP_JSON}. Generate via CLI or scripts first.")
else:
    # Expecting schema like: { "planets": { "planet_id": { "top_bins": [...], "scores": {...} } }, ...}
    planets = overlay.get("planets") or overlay  # tolerate flat schema
    print(f"Loaded overlay for {len(planets)} planets.")
    # Simple aggregate: most common bins across planets (if present)
    bin_counts = Counter()
    for pid, rec in planets.items():
        top_bins = rec.get("top_bins") or []
        bin_counts.update(top_bins)
    most_common = bin_counts.most_common(10)
    print("Top 10 recurrent bins across planets:", most_common)


## 🧩 Symbolic Rule Violations — summary view

In [ ]:
# Inspect symbolic violation summaries to find dominant rules and hotspots.
symb = safe_load_json(SYMB_JSON)
if symb is None:
    print(f"⚠️ No symbolic violation summary at {SYMB_JSON}.")
else:
    # Tolerate either list or dict formats
    if isinstance(symb, dict) and "rules" in symb:
        rules = symb["rules"]
    elif isinstance(symb, list):
        rules = symb
    else:
        rules = symb

    print("Symbolic rules summary keys:", list(rules)[:10] if isinstance(rules, dict) else "list")
    # Very simple aggregate if dict: sort rules by mean violation
    if isinstance(rules, dict):
        agg = []
        for rname, vals in rules.items():
            v = vals if isinstance(vals, (int, float)) else vals.get("mean", None) if isinstance(vals, dict) else None
            if v is not None:
                agg.append((rname, float(v)))
        agg.sort(key=lambda x: x[1], reverse=True)
        print("Top 10 rules by mean violation:")
        for r, v in agg[:10]:
            print(f"  {r:40s}  {v:.4f}")


## 📈 FFT & Smoothness — quick checks on μ spectra

In [ ]:
# Try a lightweight smoothness/FFT visualization from diagnostic_summary.json if available.
import numpy as np
import matplotlib.pyplot as plt

diag = safe_load_json(DIAG_SUMMARY)
if not diag:
    print(f"⚠️ No diagnostic summary at {DIAG_SUMMARY}. Generate via CLI or scripts first.")
else:
    # Heuristic: look for one planet entry with "mu" or "spectrum" field
    # and draw its FFT magnitude and finite-difference smoothness.
    # The exact schema varies; we attempt to be permissive.
    candidates = []
    if isinstance(diag, dict):
        # Possible structures: {"planets": {...}}, or {"items": [...]}, or direct
        root = diag.get("planets") or diag.get("items") or diag
        if isinstance(root, dict):
            candidates = list(root.values())
        elif isinstance(root, list):
            candidates = root
    if not candidates:
        print("Could not find planet entries in diagnostic summary.")
    else:
        # Find first record with a 1D mu-like array
        mu = None
        for rec in candidates:
            arr = rec.get("mu") or rec.get("spectrum") or rec.get("mu_mean")
            if isinstance(arr, list) and len(arr) >= 16:
                mu = np.array(arr, dtype=float)
                break
        if mu is None:
            print("No suitable μ array found in diagnostic summary.")
        else:
            # FFT magnitude
            fft_mag = np.abs(np.fft.rfft(mu))
            plt.figure(figsize=(6,4))
            plt.plot(fft_mag)
            plt.title("FFT magnitude of μ (example planet)")
            plt.xlabel("Frequency bin")
            plt.ylabel("|FFT|")
            plt.show()

            # Smoothness (finite differences)
            grad = np.diff(mu)
            curv = np.diff(mu, n=2)
            plt.figure(figsize=(6,4))
            plt.plot(np.abs(grad), label="|∂μ|")
            plt.plot(np.abs(curv), label="|∂²μ|")
            plt.title("Smoothness diagnostics of μ (example planet)")
            plt.xlabel("Spectral bin")
            plt.ylabel("Magnitude")
            plt.legend()
            plt.show()


## 🎯 Calibration (σ) — COREL overview

In [ ]:
# Load calibration summary if available and show simple histograms.
import numpy as np
import matplotlib.pyplot as plt

corel = safe_load_json(COREL_JSON)
if corel is None:
    print(f"⚠️ No COREL calibration summary at {COREL_JSON}.")
else:
    # Heuristic: expect per-bin coverage or residuals arrays
    cov = corel.get("coverage") or corel.get("per_bin_coverage")
    res = corel.get("residuals") or corel.get("per_bin_residual")
    if isinstance(cov, list) and len(cov) > 0:
        cov = np.array(cov, dtype=float)
        plt.figure(figsize=(6,4))
        plt.hist(cov, bins=30, alpha=0.9)
        plt.title("Coverage histogram (COREL)")
        plt.xlabel("Coverage")
        plt.ylabel("Count")
        plt.show()
    if isinstance(res, list) and len(res) > 0:
        res = np.array(res, dtype=float)
        plt.figure(figsize=(6,4))
        plt.hist(res, bins=30, alpha=0.9)
        plt.title("Residual histogram (COREL)")
        plt.xlabel("Residual")
        plt.ylabel("Count")
        plt.show()


## 🧪 Build Diagnostics Dashboard (optional)

In [ ]:
# Call the CLI to generate the full HTML diagnostics dashboard, or load an existing one.
def run_dashboard_cli():
    try:
        exe = shutil.which("spectramind")
        if not exe:
            return False, "CLI not found"
        cmd = ["spectramind", "diagnose", "dashboard",
               "--html-out", str(DASHBOARD_HTML),
               "--log-file", str(LOG_MD),
               "--open-browser", "false"]
        print("Running:", " ".join(cmd))
        subprocess.check_call(cmd, timeout=1200)
        return True, "OK"
    except Exception as e:
        return False, str(e)

ok, msg = run_dashboard_cli()
if ok:
    print(f"✅ Diagnostics dashboard generated at: {DASHBOARD_HTML}")
elif DASHBOARD_HTML.exists():
    print("⚠️ CLI skipped/failed; using existing:", DASHBOARD_HTML)
else:
    print("❌ Dashboard not available. Generate via CLI when ready.")

print("To open manually if needed:", str(DASHBOARD_HTML))


## 🧾 Append run metadata to `v50_debug_log.md`

In [ ]:
# Append a structured entry to v50_debug_log.md for auditability.
from datetime import datetime
entry = f"""### Notebook: 02_diagnostics_explainability.ipynb
- timestamp: {datetime.now().isoformat(timespec="seconds")}
- cwd: {ROOT}
- python: {platform.python_version()}
- actions:
  - env_init
  - umap_try_cli: {'exists' if UMAP_HTML.exists() else 'missing'}
  - tsne_try_cli: {'exists' if TSNE_HTML.exists() else 'missing'}
  - shap_overlay_loaded: {SHAP_JSON.exists()}
  - symbolic_summary_loaded: {SYMB_JSON.exists()}
  - corel_summary_loaded: {COREL_JSON.exists()}
  - dashboard: {'exists' if DASHBOARD_HTML.exists() else 'missing'}
"""

try:
    with open(LOG_MD, "a", encoding="utf-8") as f:
        f.write(entry + "\n")
    print(f"Appended notebook log entry to {LOG_MD}")
except Exception as e:
    print(f"⚠️ Could not append to {LOG_MD}: {e}")


## 📚 References & Next Steps

- `spectramind diagnose umap` · Generate UMAP HTML with symbolic overlays and links.
- `spectramind diagnose tsne-latents` · Interactive t‑SNE with confidence/links.
- `spectramind diagnose smoothness` · Produce smoothness maps and CSV summaries.
- `spectramind diagnose dashboard` · Unified HTML diagnostics dashboard.

**Pro tip:** Pair this notebook with `00_quickstart.ipynb` and `03_ablation_and_tuning.ipynb` for the full pipeline flow.


## 03_ablation_and_tuning.ipynb

# 🧪 SpectraMind V50 — 03 · Ablation & Tuning

**NeurIPS 2025 Ariel Data Challenge** · *CLI‑first, Hydra‑safe, reproducible*

This notebook drives **systematic ablations and hyperparameter tuning** over the SpectraMind V50 stack.
It favors the **`spectramind` CLI** (Typer) for all heavy‑lifting, and **reads artifacts** to visualize
and compare results. Every step is **deterministic where possible** and produces evidence (logs/JSON/HTML).

> Golden rule: the notebook mirrors what you'd do from the CLI and never hides transformations.


## 🎯 Objectives

- Run or resume **symbolic‑aware ablations** (smoothness, symbolic weights, entropy penalties, COREL/σ, etc.).
- Inspect and plot **GLL, RMSE/MAE, calibration, violation scores** from ablation artifacts.
- Generate **Markdown + HTML leaderboards**, and optionally a **Top‑N ZIP** bundle.
- Append run metadata to `v50_debug_log.md` for auditability.


## 🔧 Environment & Paths

In [ ]:
from pathlib import Path
import json, os, sys, platform
from datetime import datetime

ROOT = Path.cwd()
ART = ROOT / "artifacts"
ABLATE_DIR = ART / "ablation"
ABLATE_RUNS = ABLATE_DIR / "runs"
ABLATE_SUMMARY_JSON = ABLATE_DIR / "leaderboard.json"
ABLATE_SUMMARY_CSV  = ABLATE_DIR / "leaderboard.csv"
ABLATE_SUMMARY_MD   = ABLATE_DIR / "leaderboard.md"
ABLATE_SUMMARY_HTML = ABLATE_DIR / "leaderboard.html"
TOPN_ZIP            = ABLATE_DIR / "topN_bundle.zip"
LOG_MD              = ROOT / "v50_debug_log.md"

for d in [ART, ABLATE_DIR, ABLATE_RUNS]:
    d.mkdir(parents=True, exist_ok=True)

env = {
    "timestamp": datetime.now().isoformat(timespec="seconds"),
    "python": sys.version.replace("\n", " "),
    "platform": platform.platform(),
    "cwd": str(ROOT),
    "paths": {
        "ART": str(ART),
        "ABLATE_DIR": str(ABLATE_DIR),
        "ABLATE_RUNS": str(ABLATE_RUNS),
        "ABLATE_SUMMARY_JSON": str(ABLATE_SUMMARY_JSON),
        "ABLATE_SUMMARY_CSV": str(ABLATE_SUMMARY_CSV),
        "ABLATE_SUMMARY_MD": str(ABLATE_SUMMARY_MD),
        "ABLATE_SUMMARY_HTML": str(ABLATE_SUMMARY_HTML),
        "TOPN_ZIP": str(TOPN_ZIP),
        "LOG_MD": str(LOG_MD),
    }
}
print(json.dumps(env, indent=2))


## 🩺 Quick CLI sanity (optional)

In [ ]:
import shutil, subprocess

def check_cli(cmd="spectramind", args=["--version"]):
    exe = shutil.which(cmd)
    if not exe:
        print("⚠️ 'spectramind' CLI not found on PATH. Skipping CLI sanity check.")
        return {"available": False}
    try:
        out = subprocess.check_output([cmd] + args, stderr=subprocess.STDOUT, text=True, timeout=30)
        print(out)
        return {"available": True, "output": out}
    except Exception as e:
        print(f"⚠️ CLI call failed: {e}")
        return {"available": True, "error": str(e)}

cli_info = check_cli()
cli_info


## 🧰 Define Ablation Grids

Below are **example grids** focused on symbolic and physics knobs. Tune to match your configs.

- `loss.symbolic.smoothness.lambda`: smoothness penalty weight
- `loss.symbolic.asymmetry.lambda`: asymmetry penalty weight
- `loss.symbolic.nonneg.lambda`: non‑negativity hinge
- `training.scheduler.max_lr`: learning rate peak
- `training.augmentation.jitter`: FGS1 jitter augmentation on/off
- `uncertainty.corel.enable`: COREL σ calibration toggle
- `uncertainty.temperature.value`: temperature scaling
- `decoder.sigma.attn_fusion`: enable attention×symbolic fusion in σ head


In [ ]:
# You can edit these directly or build them programmatically.
AB_GRID = {
    "one_at_a_time": {
        "loss.symbolic.smoothness.lambda": [0.0, 0.01, 0.05, 0.1],
        "loss.symbolic.asymmetry.lambda":  [0.0, 0.01, 0.05],
        "loss.symbolic.nonneg.lambda":     [0.0, 0.05],
        "training.scheduler.max_lr":       [1e-4, 3e-4, 1e-3],
        "training.augmentation.jitter":    [False, True],
        "uncertainty.corel.enable":        [False, True],
        "uncertainty.temperature.value":   [1.0, 1.25, 1.5],
        "decoder.sigma.attn_fusion":       [False, True],
    },
    "cartesian": {
        # Keep cartesian grids modest to respect runtime budgets.
        "loss.symbolic.smoothness.lambda": [0.0, 0.05],
        "training.scheduler.max_lr":       [1e-4, 3e-4],
        "uncertainty.corel.enable":        [False, True],
    }
}
print(json.dumps(AB_GRID, indent=2))


## 🚀 Launch ablations via `spectramind ablate` (or load existing results)

In [ ]:
import json, subprocess, shutil, os, time

def run_ablate(mode="one-at-a-time",
               top_n=10,
               retries=1,
               parallel=2,
               md_out=None,
               html_out=None,
               csv_out=None,
               topn_zip=None):
    exe = shutil.which("spectramind")
    if not exe:
        print("⚠️ CLI not found; skipping ablation launch. You can run it in a terminal and re-run next cells.")
        return False
    cmd = [
        "spectramind", "ablate",
        "--mode", mode,
        "--retries", str(retries),
        "--parallel", str(parallel),
        "--top_n", str(top_n),
    ]
    if md_out:
        cmd += ["--md", str(md_out)]
    if html_out:
        cmd += ["--open_html", "false", "--html", str(html_out)]
    if csv_out:
        cmd += ["--csv", str(csv_out)]
    if topn_zip:
        cmd += ["--zip", str(topn_zip)]
    # Note: advanced flags (symbols, metrics, diagnostics) are provided by the CLI impl.
    print("Running:", " ".join(cmd))
    try:
        subprocess.check_call(cmd, timeout=18000)  # 5 hours cap; adjust to your infra
        return True
    except Exception as e:
        print("Ablation run failed/aborted:", e)
        return False

# Example: try a quick one-at-a-time sweep that writes leaderboard artifacts
ok = run_ablate(
    mode="one-at-a-time",
    top_n=10,
    retries=1,
    parallel=2,
    md_out=ABLATE_SUMMARY_MD,
    html_out=ABLATE_SUMMARY_HTML,
    csv_out=ABLATE_SUMMARY_CSV,
    topn_zip=TOPN_ZIP
)

if ok:
    print("✅ Ablation CLI triggered. Artifacts should appear under:", ABLATE_DIR)
else:
    print("⚠️ Skipped or failed to launch ablation via CLI.")


## 📊 Parse leaderboard & visualize top runs

In [ ]:
import json, csv, math
from pathlib import Path

def load_leaderboard(json_path=ABLATE_SUMMARY_JSON, csv_path=ABLATE_SUMMARY_CSV):
    data = None
    if json_path.exists():
        try:
            with open(json_path, "r", encoding="utf-8") as f:
                data = json.load(f)
        except Exception as e:
            print("Failed to parse JSON leaderboard:", e)
    rows = []
    if csv_path.exists():
        try:
            import pandas as pd
            df = pd.read_csv(csv_path)
            print("CSV leaderboard loaded with shape:", df.shape)
            rows = df.to_dict(orient="records")
        except Exception as e:
            print("Failed to parse CSV leaderboard via pandas:", e)
            # Fallback to Python csv
            try:
                with open(csv_path, newline="", encoding="utf-8") as f:
                    reader = csv.DictReader(f)
                    rows = list(reader)
                print("CSV leaderboard loaded via csv module:", len(rows), "rows.")
            except Exception as e2:
                print("CSV fallback failed:", e2)
    return data, rows

lb_json, lb_rows = load_leaderboard()
if lb_json is None and not lb_rows:
    print("No leaderboard artifacts found yet. Once ablations complete, re-run this cell.")
else:
    # Heuristic: find a numeric "gll" or "score" column for ranking
    import numpy as np
    import matplotlib.pyplot as plt

    # Select rows and numeric metric
    metric_key = None
    if lb_rows:
        keys = lb_rows[0].keys()
        for k in ["gll", "GLL", "score", "val_gll", "val_score", "mean_gll"]:
            if k in keys:
                metric_key = k
                break
    # Plot top 15 by metric (lower GLL is better; adjust sign if needed)
    if lb_rows and metric_key:
        # Convert values to float where possible
        def to_float(x):
            try: return float(x)
            except: return math.inf
        sorted_rows = sorted(lb_rows, key=lambda r: to_float(r.get(metric_key, math.inf)))
        top = sorted_rows[:15]
        labels = [r.get("run_id", r.get("short_id", f"r{i}")) for i, r in enumerate(top)]
        vals = [to_float(r.get(metric_key, math.inf)) for r in top]
        plt.figure(figsize=(8, 4))
        plt.bar(range(len(vals)), vals)
        plt.xticks(range(len(vals)), labels, rotation=45, ha="right")
        plt.ylabel(metric_key)
        plt.title("Top runs by leaderboard metric")
        plt.tight_layout()
        plt.show()
    else:
        print("Leaderboard parsed, but could not identify a numeric metric column to plot.")


## 🧰 Optional: rebuild leaderboard & Top‑N ZIP via CLI

In [ ]:
# If you ran custom ablations manually, you can regenerate summary artifacts here.
import shutil, subprocess

def rebuild_leaderboard(md_out=ABLATE_SUMMARY_MD, html_out=ABLATE_SUMMARY_HTML,
                        csv_out=ABLATE_SUMMARY_CSV, topn_zip=TOPN_ZIP, top_n=10):
    exe = shutil.which("spectramind")
    if not exe:
        print("⚠️ CLI not found; cannot rebuild leaderboard via CLI.")
        return False
    cmd = [
        "spectramind", "ablate", "--rebuild-only",
        "--top_n", str(top_n),
        "--md", str(md_out),
        "--open_html", "false", "--html", str(html_out),
        "--csv", str(csv_out),
        "--zip", str(topn_zip),
    ]
    print("Running:", " ".join(cmd))
    try:
        subprocess.check_call(cmd, timeout=3600)
        return True
    except Exception as e:
        print("Rebuild failed:", e)
        return False

# Example (safe to skip):
# ok = rebuild_leaderboard()
# print("Rebuild:", ok)


## 🧾 Append run metadata to `v50_debug_log.md`

In [ ]:
from datetime import datetime
entry = f"""### Notebook: 03_ablation_and_tuning.ipynb
- timestamp: {datetime.now().isoformat(timespec="seconds")}
- cwd: {ROOT}
- python: {platform.python_version()}
- actions:
  - env_init
  - ablate_cli_attempted: true
  - leaderboard_json_exists: {ABLATE_SUMMARY_JSON.exists()}
  - leaderboard_csv_exists: {ABLATE_SUMMARY_CSV.exists()}
  - leaderboard_md_exists: {ABLATE_SUMMARY_MD.exists()}
  - leaderboard_html_exists: {ABLATE_SUMMARY_HTML.exists()}
  - topN_zip_exists: {TOPN_ZIP.exists()}
"""

try:
    with open(LOG_MD, "a", encoding="utf-8") as f:
        f.write(entry + "\n")
    print(f"Appended notebook log entry to {LOG_MD}")
except Exception as e:
    print(f"⚠️ Could not append to {LOG_MD}: {e}")


## 04_leaderboard_and_submission_fixed.ipynb

# 🏁 SpectraMind V50 — 04 · Leaderboard & Submission

This notebook closes the loop for the **NeurIPS 2025 Ariel Data Challenge**:

- Aggregate **ablation/tuning** results and pick a best run
- Run **self-test** to validate the pipeline
- Generate a **submission** (μ, σ) with CLI
- Perform basic **schema checks** on the submission
- (Optional) **Upload** to Kaggle via API
- Append an entry to `v50_debug_log.md` for reproducibility

> CLI-first, Hydra-driven, DVC-aware. Cells auto-skip gracefully if CLI isn't on PATH.


## 🔧 Environment & Paths

In [ ]:
from pathlib import Path
import json, os, sys, platform, shutil
from datetime import datetime

ROOT = Path.cwd()
ART = ROOT / "artifacts"
ABLATE_DIR = ART / "ablation"
DIAG_DIR = ART / "diagnostics"
SUBMIT_DIR = ART / "submission"
SUBMIT_DIR.mkdir(parents=True, exist_ok=True)

LEADERBOARD_JSON = ABLATE_DIR / "leaderboard.json"
LEADERBOARD_CSV  = ABLATE_DIR / "leaderboard.csv"
BEST_RUN_JSON    = ABLATE_DIR / "best_run.json"   # optional
SUBMISSION_CSV   = SUBMIT_DIR / "submission.csv"
BUNDLE_ZIP       = SUBMIT_DIR / "submission_bundle.zip"
LOG_MD           = ROOT / "v50_debug_log.md"

env = {
    "timestamp": datetime.now().isoformat(timespec="seconds"),
    "python": sys.version.replace("\n", " "),
    "platform": platform.platform(),
    "cwd": str(ROOT),
    "paths": {
        "ART": str(ART),
        "ABLATE_DIR": str(ABLATE_DIR),
        "DIAG_DIR": str(DIAG_DIR),
        "SUBMIT_DIR": str(SUBMIT_DIR),
        "LEADERBOARD_JSON": str(LEADERBOARD_JSON),
        "LEADERBOARD_CSV": str(LEADERBOARD_CSV),
        "BEST_RUN_JSON": str(BEST_RUN_JSON),
        "SUBMISSION_CSV": str(SUBMISSION_CSV),
        "BUNDLE_ZIP": str(BUNDLE_ZIP),
        "LOG_MD": str(LOG_MD),
    }
}
print(json.dumps(env, indent=2))


## 🩺 CLI sanity (optional)

In [ ]:
import shutil, subprocess

def check_cli(cmd="spectramind", args=["--version"]):
    exe = shutil.which(cmd)
    if not exe:
        print("⚠️ 'spectramind' CLI not found on PATH. Notebook will still run, but CLI calls will be skipped.")
        return {"available": False}
    try:
        out = subprocess.check_output([cmd] + args, stderr=subprocess.STDOUT, text=True, timeout=30)
        print(out)
        return {"available": True, "output": out}
    except Exception as e:
        print(f"⚠️ CLI call failed: {e}")
        return {"available": True, "error": str(e)}

cli_info = check_cli()
cli_info


## 📊 Aggregate ablation & pick best run

In [ ]:
import csv, math, json
from pathlib import Path

best = None
rows = []

# Try CSV first
if LEADERBOARD_CSV.exists():
    try:
        import pandas as pd
        df = pd.read_csv(LEADERBOARD_CSV)
        print("Loaded ablation leaderboard CSV:", LEADERBOARD_CSV, "shape=", df.shape)
        # Try common metric keys, prefer lower-is-better GLL if present
        metric_key = None
        for k in ["gll", "GLL", "score", "val_gll", "val_score", "mean_gll"]:
            if k in df.columns:
                metric_key = k
                break
        if metric_key:
            # lower GLL better; if score, assume higher is better and invert for sort flag
            ascending = True if "gll" in metric_key.lower() else False
            df_sorted = df.sort_values(metric_key, ascending=ascending)
            best = df_sorted.iloc[0].to_dict()
            display_cols = [c for c in df.columns if c in ["run_id", "short_id", "config", metric_key]]
            print("Top 5 by", metric_key)
            display(df_sorted[display_cols].head(5))
        else:
            print("No known metric column found; showing head:")
            display(df.head())
    except Exception as e:
        print("CSV parse failed:", e)

# Fallback to JSON
if best is None and LEADERBOARD_JSON.exists():
    try:
        with open(LEADERBOARD_JSON, "r", encoding="utf-8") as f:
            data = json.load(f)
        # Expect list of dicts; pick by gll or score
        metric_key = None
        if isinstance(data, list) and data:
            keys = list(data[0].keys())
            for k in ["gll", "GLL", "score", "val_gll", "val_score", "mean_gll"]:
                if k in keys:
                    metric_key = k
                    break
        if metric_key:
            def sort_key(d):
                v = d.get(metric_key, math.inf)
                try: return float(v)
                except: return math.inf
            # lower gll better; if score, higher better
            reverse = False if "gll" in metric_key.lower() else True
            data_sorted = sorted(data, key=sort_key, reverse=reverse)
            best = data_sorted[0]
            print("Top 3 candidates by", metric_key)
            for i, d in enumerate(data_sorted[:3]):
                rid = d.get("run_id") or d.get("short_id") or f"r{i}"
                print(i+1, rid, metric_key, d.get(metric_key))
        else:
            print("JSON leaderboard present but no known metric. Keys:", list(data[0].keys()) if isinstance(data, list) and data else None)
    except Exception as e:
        print("JSON parse failed:", e)

# Persist best run selection (optional)
if best:
    with open(BEST_RUN_JSON, "w", encoding="utf-8") as f:
        json.dump(best, f, indent=2)
    print("Best run saved to:", BEST_RUN_JSON)
else:
    print("⚠️ No ablation leaderboard found; proceeding without an explicit best-run selection.")


## ✅ Run pipeline self-test

In [ ]:
import subprocess, shutil

def run_selftest():
    exe = shutil.which("spectramind")
    if not exe:
        print("⚠️ spectramind CLI not found; skipping self-test.")
        return False
    # Try deep test first, fallback to fast
    cmds = [
        ["spectramind", "test", "--deep"],
        ["spectramind", "test"]
    ]
    for cmd in cmds:
        try:
            print("Running:", " ".join(cmd))
            subprocess.check_call(cmd, timeout=1800)
            print("✅ Self-test passed:", " ".join(cmd))
            return True
        except Exception as e:
            print("Self-test variant failed:", e)
    print("⚠️ All self-test variants failed.")
    return False

_ = run_selftest()


## 📦 Generate submission (μ/σ) via CLI

In [ ]:
import subprocess, shutil
from pathlib import Path

def generate_submission(out_csv: Path):
    exe = shutil.which("spectramind")
    if not exe:
        print("⚠️ spectramind CLI not found; skipping submission generation.")
        return False
    out_csv.parent.mkdir(parents=True, exist_ok=True)
    cmd = ["spectramind", "submit", "--out", str(out_csv), "--include-diagnostics", "true"]
    # If BEST_RUN_JSON exists and includes a config or short_id, pass it through as override if supported
    if BEST_RUN_JSON.exists():
        try:
            import json
            best = json.loads(BEST_RUN_JSON.read_text())
            # attempt a generic override key if available
            if "config" in best and isinstance(best["config"], str):
                cmd += ["--config", best["config"]]
            elif "short_id" in best:
                cmd += ["--run-id", str(best["short_id"])]
        except Exception as e:
            print("Note: couldn't parse BEST_RUN_JSON for overrides:", e)
    print("Running:", " ".join(cmd))
    try:
        subprocess.check_call(cmd, timeout=6*3600)  # allow long run
        print("✅ Submission written:", out_csv)
        return True
    except Exception as e:
        print("❌ Submission generation failed:", e)
        return False

ok = generate_submission(SUBMISSION_CSV)
ok


## 🔎 Basic submission schema check

In [ ]:
import pandas as pd
import numpy as np

def validate_submission(path: Path):
    if not path.exists():
        print("❌ No submission file found at", path)
        return False
    try:
        df = pd.read_csv(path)
    except Exception as e:
        print("❌ CSV read failed:", e)
        return False

    print("Submission shape:", df.shape)
    # Very generic checks: non-empty, numeric columns present
    if df.empty:
        print("❌ Submission is empty.")
        return False

    # Try to detect numeric prediction columns (μ/σ); require at least a few numeric columns
    num_cols = [c for c in df.columns if np.issubdtype(df[c].dtype, np.number)]
    if len(num_cols) < 10:
        print("⚠️ Fewer than 10 numeric columns detected; ensure μ/σ columns are included.")
    else:
        # Check no NaNs in prediction columns
        nan_counts = df[num_cols].isna().sum().sum()
        if nan_counts > 0:
            print(f"⚠️ Found {nan_counts} NaNs in numeric columns; please investigate.")

    # Optional: enforce monotonic ID or required column name heuristics if known
    print("✅ Basic checks completed.")
    return True

_ = validate_submission(SUBMISSION_CSV)


## 🗜️ Bundle submission + diagnostics

In [ ]:
import zipfile

def bundle_zip(bundle: Path, submission: Path, extras: list[Path] = None):
    extras = extras or []
    with zipfile.ZipFile(bundle, "w", compression=zipfile.ZIP_DEFLATED) as z:
        if submission.exists():
            z.write(submission, submission.name)
        else:
            print("⚠️ No submission to include.")
        for p in extras:
            if p.exists():
                if p.is_file():
                    z.write(p, p.name)
                else:
                    # add directory contents flatly
                    for child in p.rglob("*"):
                        if child.is_file():
                            arc = child.relative_to(p.parent)
                            z.write(child, str(arc))
    print("Bundle written:", bundle)

extras = [DIAG_DIR] if DIAG_DIR.exists() else []
bundle_zip(BUNDLE_ZIP, SUBMISSION_CSV, extras)


## ⤴️ (Optional) Push to Kaggle via API

In [ ]:
# This cell attempts a Kaggle upload if configured.
# Prerequisites:
#  - Install kaggle:  pip install kaggle
#  - Place Kaggle credentials (~/.kaggle/kaggle.json) or set KAGGLE_USERNAME / KAGGLE_KEY env vars
#  - Set COMPETITION to the exact competition slug

import os, subprocess, shutil

COMPETITION = os.environ.get("KAGGLE_COMPETITION", "ariel-data-challenge-2025")

def kaggle_submit(csv_path: Path, message: str = "SpectraMind V50 submission"):
    if not csv_path.exists():
        print("⚠️ No CSV to submit.")
        return False
    # Try kaggle CLI
    exe = shutil.which("kaggle")
    if not exe:
        print("ℹ️ kaggle CLI not found; skipping Kaggle upload.")
        return False
    try:
        cmd = ["kaggle", "competitions", "submit", "-c", COMPETITION, "-f", str(csv_path), "-m", message]
        print("Running:", " ".join(cmd))
        subprocess.check_call(cmd, timeout=600)
        print("✅ Kaggle submission attempted.")
        return True
    except Exception as e:
        print("⚠️ Kaggle submission failed:", e)
        return False

# Disabled by default; uncomment to attempt submission
# _ = kaggle_submit(SUBMISSION_CSV, "SpectraMind V50 auto-generated submission")


## 🧾 Append run metadata to `v50_debug_log.md`

In [ ]:
from datetime import datetime

entry = f"""### Notebook: 04_leaderboard_and_submission.ipynb
- timestamp: {datetime.now().isoformat(timespec="seconds")}
- cwd: {ROOT}
- python: {platform.python_version()}
- actions:
  - ablation_leaderboard_csv: {LEADERBOARD_CSV.exists()}
  - ablation_leaderboard_json: {LEADERBOARD_JSON.exists()}
  - best_run_json: {BEST_RUN_JSON.exists()}
  - submission_csv_exists: {SUBMISSION_CSV.exists()}
  - bundle_zip_exists: {BUNDLE_ZIP.exists()}
"""

try:
    with open(LOG_MD, "a", encoding="utf-8") as f:
        f.write(entry + "\n")
    print(f"Appended notebook log entry to {LOG_MD}")
except Exception as e:
    print(f"⚠️ Could not append to {LOG_MD}: {e}")


## 05_uncertainty_calibration_and_cycle_consistency_fixed.ipynb

# 📐 SpectraMind V50 — 05 · Uncertainty Calibration & Cycle Consistency

Goals:
- Evaluate calibration quality (coverage vs. nominal for μ/σ)
- Apply **temperature scaling** (global σ scale) and optional per-bin scaling
- Emit a **calibrated submission CSV** with updated σ
- (Optional) Run a **cycle-consistency** sanity check via forward sim
- Log a **reproducibility entry** to `v50_debug_log.md`

> CLI-first and reproducibility-friendly. Cells skip gracefully if inputs/CLI aren't present.


## 🔧 Environment & Paths

In [ ]:
from pathlib import Path
import json, os, sys, platform, shutil
from datetime import datetime

ROOT = Path.cwd()
ART = ROOT / "artifacts"
DIAG_DIR = ART / "diagnostics"
SUBMIT_DIR = ART / "submission"
CAL_DIR = ART / "calibration"
CAL_DIR.mkdir(parents=True, exist_ok=True)

# Typical inputs (if present)
DIAG_SUMMARY = DIAG_DIR / "diagnostic_summary.json"  # expected to hold residual/sigma stats if generated
SUBMISSION_CSV = SUBMIT_DIR / "submission.csv"

# Outputs
CAL_REPORT_JSON = CAL_DIR / "calibration_report.json"
CAL_FACTORS_JSON = CAL_DIR / "sigma_scale_factors.json"
CALIBRATED_SUBMISSION_CSV = SUBMIT_DIR / "submission_calibrated.csv"
LOG_MD = ROOT / "v50_debug_log.md"

env = {
    "timestamp": datetime.now().isoformat(timespec="seconds"),
    "python": sys.version.replace("\n", " "),
    "platform": platform.platform(),
    "cwd": str(ROOT),
    "paths": {
        "ART": str(ART),
        "DIAG_DIR": str(DIAG_DIR),
        "SUBMIT_DIR": str(SUBMIT_DIR),
        "CAL_DIR": str(CAL_DIR),
        "DIAG_SUMMARY": str(DIAG_SUMMARY),
        "SUBMISSION_CSV": str(SUBMISSION_CSV),
        "CAL_REPORT_JSON": str(CAL_REPORT_JSON),
        "CAL_FACTORS_JSON": str(CAL_FACTORS_JSON),
        "CALIBRATED_SUBMISSION_CSV": str(CALIBRATED_SUBMISSION_CSV),
        "LOG_MD": str(LOG_MD),
    }
}
print(json.dumps(env, indent=2))


## 🩺 CLI sanity (optional)

In [ ]:
import shutil, subprocess

def check_cli(cmd="spectramind", args=["--version"]):
    exe = shutil.which(cmd)
    if not exe:
        print("ℹ️ 'spectramind' CLI not found on PATH. CLI-dependent steps will be skipped.")
        return {"available": False}
    try:
        out = subprocess.check_output([cmd] + args, stderr=subprocess.STDOUT, text=True, timeout=30)
        print(out)
        return {"available": True, "output": out}
    except Exception as e:
        print(f"⚠️ CLI call failed: {e}")
        return {"available": True, "error": str(e)}

cli_info = check_cli()
cli_info


## 📊 Load diagnostics & compute temperature scaling

In [ ]:
import json, math
from statistics import median
import numpy as np

def load_diag_summary(path: Path):
    if not path.exists():
        print("⚠️ No diagnostic summary found:", path)
        return None
    try:
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        return data
    except Exception as e:
        print("⚠️ Failed to parse diagnostic summary:", e)
        return None

def extract_resid_sigma(diag_data):
    """Attempt to extract arrays of residuals and predicted sigmas from a generic diagnostic JSON.
    Heuristics handle a few common schemas.
    Returns arrays flattened across items/bins if possible.
    """
    resids = []
    sigmas = []
    if diag_data is None:
        return np.array([]), np.array([])

    # Common shapes: {"items": [{"residuals":[...], "sigma":[...]} ...]}
    # Or {"planets": {"id": {"residuals":[...], "sigma":[...]}}}
    # Or flat: {"residuals":[...], "sigma":[...]}
    if isinstance(diag_data, dict):
        if "items" in diag_data and isinstance(diag_data["items"], list):
            for rec in diag_data["items"]:
                r = rec.get("residuals") or rec.get("resid") or rec.get("residual")
                s = rec.get("sigma") or rec.get("sigmas") or rec.get("pred_sigma")
                if isinstance(r, list) and isinstance(s, list):
                    m = min(len(r), len(s))
                    resids.extend(r[:m])
                    sigmas.extend(s[:m])
        elif "planets" in diag_data and isinstance(diag_data["planets"], dict):
            for _, rec in diag_data["planets"].items():
                r = rec.get("residuals") or rec.get("resid") or rec.get("residual")
                s = rec.get("sigma") or rec.get("sigmas") or rec.get("pred_sigma")
                if isinstance(r, list) and isinstance(s, list):
                    m = min(len(r), len(s))
                    resids.extend(r[:m])
                    sigmas.extend(s[:m])
        else:
            r = diag_data.get("residuals") or diag_data.get("resid") or diag_data.get("residual")
            s = diag_data.get("sigma") or diag_data.get("sigmas") or diag_data.get("pred_sigma")
            if isinstance(r, list) and isinstance(s, list):
                m = min(len(r), len(s))
                resids.extend(r[:m])
                sigmas.extend(s[:m])

    res = np.array(resids, dtype=float) if resids else np.array([])
    sg = np.array(sigmas, dtype=float) if sigmas else np.array([])
    print("Extracted residuals:", res.shape, "sigmas:", sg.shape)
    return res, sg

def compute_temperature_scaling(res, sg, eps=1e-12):
    """Compute a global scale alpha for sigma to improve calibration.
    Two estimates:
      - RMS-based: sqrt(mean(res^2) / mean(sigma^2))
      - Median-abs-based: median(|res|/sigma) / median(|N(0,1)|) with median(|Z|)=0.674489...
    Returns dict with both and a chosen alpha.
    """
    if res.size == 0 or sg.size == 0 or res.size != sg.size:
        return {"alpha_rms": 1.0, "alpha_med": 1.0, "alpha": 1.0, "note": "insufficient data"}

    rms_res = np.sqrt(np.mean(res**2))
    rms_sig = np.sqrt(np.mean((sg+eps)**2))
    alpha_rms = (rms_res / (rms_sig + eps)) if rms_sig > 0 else 1.0

    ratio = np.abs(res) / (sg + eps)
    med_ratio = np.median(ratio)
    med_abs_z = 0.6744897501960817  # median |N(0,1)|
    alpha_med = (med_ratio / med_abs_z) if med_abs_z > 0 else 1.0

    # Choose alpha preferring robust median, falling back to rms if extreme
    alpha = alpha_med if 0.1 <= alpha_med <= 10 else alpha_rms
    return {"alpha_rms": float(alpha_rms), "alpha_med": float(alpha_med), "alpha": float(alpha)}

def nominal_coverage_checks(res, sg, alphas=(1.0,), eps=1e-12):
    """Compute empirical coverage for ±1σ (~68.27%) and ±1.96σ (~95%) under various alphas."""
    if res.size == 0 or sg.size == 0 or res.size != sg.size:
        return {}
    out = {}
    for a in alphas:
        z = np.abs(res) / (a*(sg+eps))
        cov68 = float(np.mean(z <= 1.0))
        cov95 = float(np.mean(z <= 1.96))
        out[str(a)] = {"cov_68": cov68, "cov_95": cov95}
    return out

diag = load_diag_summary(DIAG_SUMMARY)
res, sg = extract_resid_sigma(diag)
scales = compute_temperature_scaling(res, sg)
cov = nominal_coverage_checks(res, sg, alphas=(1.0, scales.get("alpha", 1.0)))
print("Proposed scaling:", json.dumps(scales, indent=2))
print("Coverage:", json.dumps(cov, indent=2))

# Persist a small calibration report
report = {
    "timestamp": datetime.now().isoformat(timespec="seconds"),
    "counts": int(min(res.size, sg.size)),
    "scales": scales,
    "coverage": cov
}
with open(CAL_REPORT_JSON, "w", encoding="utf-8") as f:
    json.dump(report, f, indent=2)

with open(CAL_FACTORS_JSON, "w", encoding="utf-8") as f:
    json.dump({"global_sigma_scale": scales.get("alpha", 1.0)}, f, indent=2)

print("Saved:", CAL_REPORT_JSON, "and", CAL_FACTORS_JSON)


## 📈 Visualize calibration (optional)

In [ ]:
# Build an empirical CDF of |res|/sigma and overlay nominal 68/95 thresholds
import numpy as np
import matplotlib.pyplot as plt

def plot_empirical_cdf(res, sg, alpha=1.0, title="Empirical |z| CDF"):
    if res.size == 0 or sg.size == 0 or res.size != sg.size:
        print("No data to plot.")
        return
    z = np.abs(res) / (alpha*(sg+1e-12))
    z_sorted = np.sort(z)
    y = np.linspace(0, 1, len(z_sorted), endpoint=False)

    plt.figure(figsize=(6,4))
    plt.plot(z_sorted, y)
    plt.axvline(1.0, linestyle="--")
    plt.axvline(1.96, linestyle="--")
    plt.title(title)
    plt.xlabel("|z| = |res| / (alpha * sigma)")
    plt.ylabel("CDF")
    plt.show()

# Raw and scaled views (if data available)
if res.size and sg.size and res.size == sg.size:
    plot_empirical_cdf(res, sg, alpha=1.0, title="Empirical |z| CDF (alpha=1.0)")
    plot_empirical_cdf(res, sg, alpha=max(1e-6, float(scales.get("alpha", 1.0))), title=f"Empirical |z| CDF (alpha={scales.get('alpha', 1.0):.3f})")
else:
    print("⚠️ Skipping plots — residual/sigma arrays unavailable.")


## 🛠️ Apply scaling to submission σ columns and save calibrated CSV

In [ ]:
import pandas as pd
import numpy as np

def detect_sigma_columns(df: pd.DataFrame):
    # Heuristics: columns containing "sigma" or ending with "_sigma"
    cands = [c for c in df.columns if "sigma" in c.lower() or c.lower().endswith("_sigma")]
    return cands

def scale_submission_sigmas(sub_csv: Path, out_csv: Path, alpha: float):
    if not sub_csv.exists():
        print("⚠️ Submission not found:", sub_csv)
        return False, []
    try:
        df = pd.read_csv(sub_csv)
    except Exception as e:
        print("❌ Failed to read submission CSV:", e)
        return False, []

    sigma_cols = detect_sigma_columns(df)
    if not sigma_cols:
        # Fallback: if columns alternate mu/sigma per bin with patterns, user can adapt here
        print("⚠️ No sigma-like columns detected; no scaling applied.")
        out_csv.write_text(df.to_csv(index=False))
        return True, []

    alpha = float(alpha) if np.isfinite(alpha) else 1.0
    df[sigma_cols] = df[sigma_cols].astype(float) * alpha
    out_csv.write_text(df.to_csv(index=False))
    print(f"✅ Wrote calibrated submission with alpha={alpha} to", out_csv)
    return True, sigma_cols

alpha = float((scales or {}).get("alpha", 1.0))
ok, used_cols = scale_submission_sigmas(SUBMISSION_CSV, CALIBRATED_SUBMISSION_CSV, alpha)
print("Sigma columns scaled:", used_cols)


## 🔬 (Optional) Per-bin scaling (if binwise stats available)

In [ ]:
# If DIAG_SUMMARY contains per-bin RMSE and mean sigma per wavelength/bin,
# compute per-bin alpha and (optionally) save a separate per-bin-calibrated submission.
import numpy as np
import pandas as pd

PERBIN_FACTORS_JSON = CAL_DIR / "sigma_scale_perbin.json"
CALIBRATED_PERBIN_SUBMISSION_CSV = SUBMIT_DIR / "submission_calibrated_perbin.csv"

def compute_perbin_alphas(diag_data):
    # Heuristics: look for entries like {"bin_index": i, "rmse": ..., "mean_sigma": ...}
    # or arrays diag_data["per_bin"]["rmse"], diag_data["per_bin"]["mean_sigma"]
    if diag_data is None:
        return None
    rmse = None; msig = None
    if isinstance(diag_data, dict) and "per_bin" in diag_data:
        per = diag_data["per_bin"]
        if isinstance(per, dict):
            rmse = per.get("rmse")
            msig = per.get("mean_sigma") or per.get("sigma_mean")
    if isinstance(rmse, list) and isinstance(msig, list) and len(rmse)==len(msig):
        rmse = np.array(rmse, dtype=float)
        msig = np.array(msig, dtype=float)
        with np.errstate(divide='ignore', invalid='ignore'):
            alphas = np.where(msig>0, np.sqrt(rmse**2 / (msig**2 + 1e-12)), 1.0)
        return alphas.tolist()
    return None

perbin_alphas = compute_perbin_alphas(diag)
if perbin_alphas:
    with open(PERBIN_FACTORS_JSON, "w", encoding="utf-8") as f:
        json.dump({"per_bin_alphas": perbin_alphas}, f, indent=2)
    print("Saved per-bin scale factors:", PERBIN_FACTORS_JSON)

    # Attempt to apply to submission if it has distinct sigma columns per-bin
    try:
        df = pd.read_csv(SUBMISSION_CSV)
        sigma_cols = [c for c in df.columns if "sigma" in c.lower() or c.lower().endswith("_sigma")]
        # If sigma columns equal in number to perbin_alphas, map directly
        if sigma_cols and len(sigma_cols) == len(perbin_alphas):
            df[sigma_cols] = df[sigma_cols].astype(float) * np.array(perbin_alphas, dtype=float)
            CALIBRATED_PERBIN_SUBMISSION_CSV.write_text(df.to_csv(index=False))
            print("✅ Wrote per-bin calibrated submission:", CALIBRATED_PERBIN_SUBMISSION_CSV)
        else:
            print("ℹ️ Per-bin scaling not applied: mismatch between sigma columns and per-bin factors.")
    except Exception as e:
        print("ℹ️ Per-bin scaling skipped due to error:", e)
else:
    print("ℹ️ No per-bin stats detected; skipping per-bin scaling.")


## 🔁 (Optional) Cycle-consistency via forward simulation

In [ ]:
import shutil, subprocess

def run_cycle_consistency(sub_csv: Path):
    exe = shutil.which("spectramind")
    if not exe:
        print("ℹ️ spectramind CLI not found; skipping forward-sim cycle test.")
        return False
    # Hypothetical CLI signature; adjust to your repo's `simulate` subcommand if present
    cmd = ["spectramind", "simulate", "--from-spectra", str(sub_csv), "--out", str(CAL_DIR / "simulated_observations")]
    print("Running:", " ".join(cmd))
    try:
        subprocess.check_call(cmd, timeout=3600)
        print("✅ Forward simulation produced artifacts in:", CAL_DIR / "simulated_observations")
        return True
    except Exception as e:
        print("ℹ️ Forward-sim step failed or not implemented:", e)
        return False

# Try on calibrated submission if present; fallback to original
target_csv = CALIBRATED_SUBMISSION_CSV if CALIBRATED_SUBMISSION_CSV.exists() else SUBMISSION_CSV
_ = run_cycle_consistency(target_csv) if target_csv.exists() else print("ℹ️ No submission CSV available for cycle check.")


## 🧾 Append run metadata to `v50_debug_log.md`

In [ ]:
from datetime import datetime

entry = f"""### Notebook: 05_uncertainty_calibration_and_cycle_consistency.ipynb
- timestamp: {datetime.now().isoformat(timespec="seconds")}
- cwd: {ROOT}
- python: {platform.python_version()}
- actions:
  - diag_summary_present: {DIAG_SUMMARY.exists()}
  - submission_csv_present: {SUBMISSION_CSV.exists()}
  - alpha_used: {(scales or {}).get("alpha", 1.0)}
  - calibrated_submission_csv_exists: {CALIBRATED_SUBMISSION_CSV.exists()}
"""
try:
    with open(LOG_MD, "a", encoding="utf-8") as f:
        f.write(entry + "\n")
    print(f"Appended notebook log entry to {LOG_MD}")
except Exception as e:
    print(f"⚠️ Could not append to {LOG_MD}: {e}")


## 06_physics_informed_modeling_and_symbolic_constraints.ipynb

# 🛰️ SpectraMind V50 — Physics‑Informed Modeling & Symbolic Constraints (Notebook 06)

**Purpose.** Add *physics‑informed, symbolic constraints* to the SpectraMind V50 pipeline and run diagnostics for
constraint violations and cycle‑consistency. The notebook adheres to the CLI‑first, Hydra‑safe workflow used in 00–05.

**Sections**
1. Pre‑flight & environment capture
2. Compose Hydra overrides for symbolic losses
3. Train with symbolic constraints
4. Symbolic diagnostics (rule ranking/overlays)
5. Cycle‑consistency (simulate μ → validate)
6. Artifacts & next steps

> Degrades gracefully: if the `spectramind` CLI is not available, the notebook switches to **DRY‑RUN** and still produces configs/logs/placeholder artifacts to keep the workflow reproducible.


In [ ]:
# ░░ Pre‑flight: environment, run IDs, paths, CLI detection ░░
import os, sys, json, platform, shutil, subprocess, datetime, pathlib

RUN_TS = datetime.datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
RUN_ID = f"physics_informed_{RUN_TS}"
ROOT_OUT = "/mnt/data/physics_informed"
ARTIFACTS = os.path.join(ROOT_OUT, RUN_ID)
LOGS = os.path.join(ARTIFACTS, "logs")
CFG_OUT = os.path.join(ARTIFACTS, "configs")
DIAG_OUT = os.path.join(ARTIFACTS, "diagnostics")
SIM_OUT = os.path.join(ARTIFACTS, "simulation")
for p in (ROOT_OUT, ARTIFACTS, LOGS, CFG_OUT, DIAG_OUT, SIM_OUT):
    os.makedirs(p, exist_ok=True)

def which(cmd: str):
    return shutil.which(cmd) is not None

CLI_PRESENT = which("spectramind")
DRY_RUN = not CLI_PRESENT

def git_cmd(args):
    try:
        out = subprocess.check_output(["git", *args], stderr=subprocess.STDOUT, timeout=5).decode().strip()
        return out
    except Exception:
        return None

env = {
    "python": sys.version.replace("\n"," "),
    "platform": platform.platform(),
    "cli_present": CLI_PRESENT,
    "dry_run": DRY_RUN,
    "run_id": RUN_ID,
    "paths": {"artifacts": ARTIFACTS, "logs": LOGS, "configs": CFG_OUT, "diagnostics": DIAG_OUT, "simulation": SIM_OUT},
    "git": {
        "commit": git_cmd(["rev-parse", "HEAD"]),
        "branch": git_cmd(["rev-parse", "--abbrev-ref", "HEAD"]),
        "status": git_cmd(["status", "--porcelain"]),
    },
}
with open(os.path.join(ARTIFACTS, "env.json"), "w") as f:
    json.dump(env, f, indent=2)

print("=== SpectraMind V50 — Notebook 06 ===")
print(json.dumps(env, indent=2))


## Configuration knobs (Hydra overrides)

**Symbolic losses** enabled here:
- `nonnegativity` — penalize negative flux/μ
- `smoothness` — L2 gradient/curvature prior on μ
- `fft_coherence` — spectral structure coherence
- `molecular_priors` — optional rule pack (H₂O/CO₂/CH₄ bands)

> Start with small weights and increase gradually while monitoring GLL and violation dashboards.


In [ ]:
import json, os

overrides = {
    "loss.symbolic.enable": "true",
    "loss.symbolic.weights.nonnegativity": "1.0",
    "loss.symbolic.weights.smoothness": "0.05",
    "loss.symbolic.weights.fft_coherence": "0.10",
    "loss.symbolic.molecular_priors.enable": "true",
    "loss.symbolic.molecular_priors.pack": "default_v1",
    "diagnostics.symbolic.top_k": "12",
    "training.max_epochs": "12",
    "training.batch_size": "16",
    "data": "ariel_nominal",
    "model": "v50",
    "training.seed": "1337",
}

cfg_file = os.path.join(CFG_OUT, "symbolic_overrides.json")
with open(cfg_file, "w") as f:
    json.dump(overrides, f, indent=2)

print("Saved overrides ->", cfg_file)
print(json.dumps(overrides, indent=2))


## Helper: robust CLI runner (uses DRY‑RUN when CLI not present)

In [ ]:
import shlex, subprocess, time

def run_cli(cmd_list, log_name="run"):
    log_path = os.path.join(LOGS, f"{log_name}.log")
    err_path = os.path.join(LOGS, f"{log_name}.err")
    start = time.time()
    result = {"cmd": cmd_list, "dry_run": DRY_RUN, "returncode": 0, "stdout": "", "stderr": ""}
    if DRY_RUN:
        msg = f"[DRY-RUN] Would execute: {' '.join(shlex.quote(c) for c in cmd_list)}\n"
        result["stdout"] = msg
        with open(log_path, "w") as f: f.write(msg)
        with open(err_path, "w") as f: f.write("")
        placeholder = os.path.join(ARTIFACTS, "dry_run_placeholder.txt")
        with open(placeholder, "a") as f: f.write(msg)
        return result

    with open(log_path, "wb") as out, open(err_path, "wb") as err:
        try:
            proc = subprocess.Popen(cmd_list, stdout=out, stderr=err, env=os.environ.copy())
            proc.wait()
            result["returncode"] = proc.returncode
        except Exception as e:
            result["returncode"] = 99
            with open(err_path, "ab") as errf:
                errf.write(str(e).encode())

    try:
        result["stdout"] = open(log_path, "r").read()
    except Exception:
        pass
    try:
        result["stderr"] = open(err_path, "r").read()
    except Exception:
        pass
    result["elapsed_sec"] = round(time.time() - start, 3)
    print(f"[rc={result['returncode']}] logs: {log_path}")
    return result


## Train with physics‑informed symbolic constraints

In [ ]:
cmd = [
    "spectramind", "train",
    "--config-name", "config_v50.yaml",
    "+outputs.root_dir=" + ARTIFACTS,
]
for k, v in overrides.items():
    cmd.append(f"+{k}={v}")
cmd += ["+training.fast_mode=true"]  # if supported

res_train = run_cli(cmd, log_name="01_train_symbolic")
print(res_train["stdout"][:500])
if res_train["returncode"] not in (0, None):
    print("Training non-zero return code:", res_train["returncode"])


## Symbolic diagnostics & rule ranking

In [ ]:
cmd_diag = [
    "spectramind", "diagnose", "symbolic-rank",
    "--top-k", overrides.get("diagnostics.symbolic.top_k", "12"),
    "--export", DIAG_OUT,
]
res_diag = run_cli(cmd_diag, log_name="02_diagnose_symbolic_rank")
print(res_diag["stdout"][:500])

cmd_dash = [
    "spectramind", "diagnose", "dashboard",
    "--out", os.path.join(DIAG_OUT, "diagnostic_report_v1.html"),
    "--show-logic-graph",
]
res_dash = run_cli(cmd_dash, log_name="03_diagnose_dashboard")
print(res_dash["stdout"][:500])


## Cycle‑consistency: simulate → validate

In [ ]:
# Prepare a tiny placeholder μ CSV if none exists (DRY-RUN friendly)
mu_csv = os.path.join(ARTIFACTS, "pred_mu.csv")
if not os.path.exists(mu_csv):
    import csv
    with open(mu_csv, "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["planet_id"] + [f"mu_{i:03d}" for i in range(283)])
        for pid in ["P0001","P0002","P0003"]:
            row = [pid] + [0.0]*283
            w.writerow(row)

cmd_sim = ["spectramind", "simulate-lightcurve-from-mu", "--mu-csv", mu_csv, "--out", SIM_OUT]
res_sim = run_cli(cmd_sim, log_name="04_simulate_from_mu")
print(res_sim["stdout"][:500])

cmd_cc = ["spectramind", "validate", "cycle-consistency",
          "--sim-dir", SIM_OUT, "--mu-ref", mu_csv,
          "--out", os.path.join(DIAG_OUT, "cycle_consistency.json")]
res_cc = run_cli(cmd_cc, log_name="05_cycle_consistency")
print(res_cc["stdout"][:500])


## Browse produced artifacts

In [ ]:
import os

def tree(path, prefix=""):
    items = sorted(os.listdir(path))
    lines = []
    for i, name in enumerate(items):
        full = os.path.join(path, name)
        connector = "└── " if i == len(items)-1 else "├── "
        lines.append(prefix + connector + name)
        if os.path.isdir(full):
            extension = "    " if i == len(items)-1 else "│   "
            lines.extend(tree(full, prefix + extension))
    return lines

print("ARTIFACTS TREE:", ARTIFACTS)
print("\n".join(tree(ARTIFACTS)))
dash_path = os.path.join(DIAG_OUT, "diagnostic_report_v1.html")
print("\nDashboard:", dash_path if os.path.exists(dash_path) else "(not found)")


## Pipeline sketch (Mermaid)

```mermaid
flowchart LR
  A[Calibrated data] --> B[Train with symbolic losses]
  B --> C[Predict μ, σ]
  C --> D[Symbolic diagnostics<br/>rule ranking & overlays]
  C --> E[Simulate lightcurves from μ]
  E --> F[Cycle‑consistency validation]
  D --> G[Dashboard / Reports]
  F --> G
```

## Next steps
- Sweep symbolic weights via `spectramind ablate` and compare GLL vs. violation score.
- Enable molecule‑specific prior packs where available; monitor per‑band improvements.
- Integrate outputs into your unified HTML report and CI for regression checks.


## 07_huggingface_integration_and_transfer_learning.ipynb

# 🤝 SpectraMind V50 — Hugging Face Integration & Transfer Learning (Notebook 07)

**Goal.** Integrate **Hugging Face** models into the SpectraMind V50 pipeline and demonstrate **parameter‑efficient fine‑tuning (PEFT/LoRA)** in a CLI‑first, Hydra‑safe flow. This notebook follows the physics‑informed work in 06 and extends the pipeline with pretrained backbones and transfer learning.

**What you’ll do**
1. Pre‑flight & environment capture (CLI presence, run IDs)
2. HF environment check (Transformers / Accelerate / PEFT) and graceful fallbacks
3. Compose Hydra overrides to select a **HF model** (e.g., ViT/TimeSformer) and **LoRA** knobs
4. Train via `spectramind train` with HF + PEFT overrides
5. Diagnose & compare against the custom SSM+GNN baseline
6. Artifact tree, Mermaid sketch, and next steps

> The notebook **degrades gracefully**: if `spectramind` or HF libs are not installed, it runs **DRY‑RUN** and still produces configs/logs/placeholder artifacts.


In [ ]:
# ░░ Pre-flight: env, run IDs, paths, CLI detection ░░
import os, sys, json, platform, shutil, subprocess, datetime, pathlib

RUN_TS = datetime.datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
RUN_ID = f"huggingface_transfer_{RUN_TS}"
ROOT_OUT = "/mnt/data/hf_transfer"
ARTIFACTS = os.path.join(ROOT_OUT, RUN_ID)
LOGS = os.path.join(ARTIFACTS, "logs")
CFG_OUT = os.path.join(ARTIFACTS, "configs")
DIAG_OUT = os.path.join(ARTIFACTS, "diagnostics")
for p in (ROOT_OUT, ARTIFACTS, LOGS, CFG_OUT, DIAG_OUT):
    os.makedirs(p, exist_ok=True)

def which(cmd: str) -> bool:
    return shutil.which(cmd) is not None

CLI_PRESENT = which("spectramind")
DRY_RUN = not CLI_PRESENT

def git_cmd(args):
    try:
        out = subprocess.check_output(["git", *args], stderr=subprocess.STDOUT, timeout=5).decode().strip()
        return out
    except Exception:
        return None

env = {
    "python": sys.version.replace("\n", " "),
    "platform": platform.platform(),
    "cli_present": CLI_PRESENT,
    "dry_run": DRY_RUN,
    "run_id": RUN_ID,
    "paths": {"artifacts": ARTIFACTS, "logs": LOGS, "configs": CFG_OUT, "diagnostics": DIAG_OUT},
    "git": {
        "commit": git_cmd(["rev-parse", "HEAD"]),
        "branch": git_cmd(["rev-parse", "--abbrev-ref", "HEAD"]),
        "status": git_cmd(["status", "--porcelain"]),
    },
}
with open(os.path.join(ARTIFACTS, "env.json"), "w") as f:
    json.dump(env, f, indent=2)

print("=== SpectraMind V50 — Notebook 07 ===")
print(json.dumps(env, indent=2))


## Hugging Face environment check (Transformers / Accelerate / PEFT)

We try importing `transformers`, `accelerate`, and `peft`. If any are missing, we **don’t install** them here (to keep the notebook reproducible/air‑gapped) but proceed in **DRY‑RUN**.


In [ ]:
missing = []
try:
    import transformers  # type: ignore
    tr_ok = True
except Exception:
    tr_ok = False
    missing.append("transformers")
try:
    import accelerate  # type: ignore
    acc_ok = True
except Exception:
    acc_ok = False
    missing.append("accelerate")
try:
    import peft  # type: ignore
    peft_ok = True
except Exception:
    peft_ok = False
    missing.append("peft")

print("HF libs — transformers:", tr_ok, "| accelerate:", acc_ok, "| peft:", peft_ok)
if missing:
    print("[NOTE] Missing libs:", ", ".join(missing), " — continuing with DRY‑RUN semantics where required.")


## Compose Hydra overrides: select HF backbone + LoRA

Common toggles:
- `model=hf_vit` or `model=hf_timesformer` (assumes your repo has these config groups)
- `peft.lora.enable=true`, with rank/alpha/dropout as tunables
- Batch/epochs kept conservative by default; use ablations later for sweeps


In [ ]:
import json, os

# Sensible defaults (adjust to your config structure)
overrides = {
    # Switch to an HF model group (example names; match to your repo’s configs/)
    "model": "hf_vit",                 # or "hf_timesformer"
    # Enable PEFT/LoRA adapters
    "peft.lora.enable": "true",
    "peft.lora.r": "16",
    "peft.lora.alpha": "32",
    "peft.lora.dropout": "0.05",
    # Data and training
    "data": "ariel_nominal",
    "training.max_epochs": "6",
    "training.batch_size": "16",
    "training.seed": "1337",
    # Optional: mixed precision / accelerate flags if your code supports these hydra keys
    "training.mixed_precision": "fp16",
}

cfg_file = os.path.join(CFG_OUT, "hf_peft_overrides.json")
with open(cfg_file, "w") as f:
    json.dump(overrides, f, indent=2)
print("Saved overrides ->", cfg_file)
print(json.dumps(overrides, indent=2))


## Helper: robust CLI runner (DRY‑RUN when CLI not present)

In [ ]:
import shlex, time

def run_cli(cmd_list, log_name="run"):
    log_path = os.path.join(LOGS, f"{log_name}.log")
    err_path = os.path.join(LOGS, f"{log_name}.err")
    start = time.time()
    result = {"cmd": cmd_list, "dry_run": DRY_RUN, "returncode": 0, "stdout": "", "stderr": ""}
    if DRY_RUN:
        msg = f"[DRY‑RUN] Would execute: {' '.join(shlex.quote(c) for c in cmd_list)}\n"
        result["stdout"] = msg
        with open(log_path, "w") as f: f.write(msg)
        with open(err_path, "w") as f: f.write("")
        place = os.path.join(ARTIFACTS, "dry_run_placeholder.txt")
        with open(place, "a") as f: f.write(msg)
        return result

    with open(log_path, "wb") as out, open(err_path, "wb") as err:
        try:
            proc = subprocess.Popen(cmd_list, stdout=out, stderr=err, env=os.environ.copy())
            proc.wait()
            result["returncode"] = proc.returncode
        except Exception as e:
            result["returncode"] = 99
            with open(err_path, "ab") as errf:
                errf.write(str(e).encode())

    try:
        result["stdout"] = open(log_path, "r").read()
    except Exception:
        pass
    try:
        result["stderr"] = open(err_path, "r").read()
    except Exception:
        pass
    result["elapsed_sec"] = round(time.time() - start, 3)
    print(f"[rc={result['returncode']}] logs: {log_path}")
    return result


## Train with Hugging Face + PEFT/LoRA (CLI‑first)

We pass the overrides to `spectramind train` and let Hydra compose the full config. This aligns with the CLI‑first/hydra architecture and reproducibility practices.


In [ ]:
cmd = [
    "spectramind", "train",
    "--config-name", "config_v50.yaml",
    "+outputs.root_dir=" + ARTIFACTS,
]
for k, v in overrides.items():
    cmd.append(f"+{k}={v}")
# Optional fast mode, if supported by your code:
cmd += ["+training.fast_mode=true"]

res_train = run_cli(cmd, log_name="01_train_hf_peft")
print(res_train["stdout"][:500])
if res_train["returncode"] not in (0, None):
    print("Training non-zero return code:", res_train["returncode"])


## Diagnostics & comparison vs custom SSM+GNN

We run diagnostics and (optionally) a comparison routine (e.g., validation metrics table or overlay plots) to quantify transfer benefits.


In [ ]:
# Symbolic / general diagnostics (adjust to your CLI):
cmd_diag = ["spectramind", "diagnose", "dashboard",
            "--out", os.path.join(DIAG_OUT, "diagnostic_report_hf_v1.html")]
res_diag = run_cli(cmd_diag, log_name="02_diagnose_dashboard")
print(res_diag["stdout"][:500])

# (Optional) If you have a comparison subcommand:
cmd_cmp = ["spectramind", "diagnose", "compare",
           "--models", "baseline_v50,hf_vit",
           "--out", os.path.join(DIAG_OUT, "compare_baseline_vs_hf.json")]
res_cmp = run_cli(cmd_cmp, log_name="03_diagnose_compare")
print(res_cmp["stdout"][:500])


## Browse produced artifacts

In [ ]:
import os

def tree(path, prefix=""):
    items = sorted(os.listdir(path))
    lines = []
    for i, name in enumerate(items):
        full = os.path.join(path, name)
        connector = "└── " if i == len(items)-1 else "├── "
        lines.append(prefix + connector + name)
        if os.path.isdir(full):
            extension = "    " if i == len(items)-1 else "│   "
            lines.extend(tree(full, prefix + extension))
    return lines

print("ARTIFACTS TREE:", ARTIFACTS)
print("\n".join(tree(ARTIFACTS)))
dash_path = os.path.join(DIAG_OUT, "diagnostic_report_hf_v1.html")
print("\nDashboard:", dash_path if os.path.exists(dash_path) else "(not found)")


## Pipeline sketch (Mermaid)

```mermaid
flowchart LR
  A[Pretrained HF backbone] --> B[PEFT/LoRA fine‑tune (Hydra)]
  B --> C[Predict μ, σ]
  C --> D[Diagnostics & overlays]
  D --> E[Compare vs SSM+GNN baseline]
  E --> F[Report / CI]
```

## Next steps
- Try **TimeSformer** or additional HF backbones; tune LoRA rank/alpha/dropout via `spectramind ablate`.
- Enable **Accelerate** multi‑GPU or mixed precision for faster runs if supported by your config.
- Publish the fine‑tuned model to a private registry (or Hugging Face Hub) for controlled reuse.


## 08_ensembles_mc_dropout_and_corel_conformal.ipynb

# 📈 SpectraMind V50 — Ensembles, MC Dropout & COREL Conformal (Notebook 08)

**Goal.** Add **epistemic uncertainty** and **coverage-guaranteed intervals** on top of per-bin σ by:
- Training **ensembles** (multi-seed or multi-arch)
- Enabling **MC Dropout** at inference
- Running **post-hoc calibration** (temperature scaling; optional per-wavelength)
- Applying **COREL conformal prediction** for graph-aware coverage

**Workflow**
1. Pre-flight & environment capture
2. Hydra overrides for ensembles, MC dropout, calibration, and COREL
3. Train K ensemble members (looped CLI calls)
4. Predict per member; aggregate mean/variance
5. Temperature scaling (post-hoc)
6. COREL conformal calibration & coverage evaluation
7. Diagnostics dashboard + artifact tree

> As with earlier notebooks, this is **CLI-first & Hydra-safe** with **DRY-RUN** fallbacks when `spectramind` is unavailable.


In [ ]:
# ░░ Pre-flight: env, run IDs, paths, CLI detection ░░
import os, sys, json, platform, shutil, subprocess, datetime, pathlib

RUN_TS = datetime.datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
RUN_ID = f"uq_conformal_{RUN_TS}"
ROOT_OUT = "/mnt/data/uq_conformal"
ARTIFACTS = os.path.join(ROOT_OUT, RUN_ID)
LOGS = os.path.join(ARTIFACTS, "logs")
CFG_OUT = os.path.join(ARTIFACTS, "configs")
PRED_OUT = os.path.join(ARTIFACTS, "predictions")
DIAG_OUT = os.path.join(ARTIFACTS, "diagnostics")
for p in (ROOT_OUT, ARTIFACTS, LOGS, CFG_OUT, PRED_OUT, DIAG_OUT):
    os.makedirs(p, exist_ok=True)

def which(cmd: str) -> bool:
    return shutil.which(cmd) is not None

CLI_PRESENT = which("spectramind")
DRY_RUN = not CLI_PRESENT

def git_cmd(args):
    try:
        out = subprocess.check_output(["git", *args], stderr=subprocess.STDOUT, timeout=5).decode().strip()
        return out
    except Exception:
        return None

env = {
    "python": sys.version.replace("\n", " "),
    "platform": platform.platform(),
    "cli_present": CLI_PRESENT,
    "dry_run": DRY_RUN,
    "run_id": RUN_ID,
    "paths": {"artifacts": ARTIFACTS, "logs": LOGS, "configs": CFG_OUT, "predictions": PRED_OUT, "diagnostics": DIAG_OUT},
    "git": {
        "commit": git_cmd(["rev-parse", "HEAD"]),
        "branch": git_cmd(["rev-parse", "--abbrev-ref", "HEAD"]),
        "status": git_cmd(["status", "--porcelain"]),
    },
}
with open(os.path.join(ARTIFACTS, "env.json"), "w") as f:
    json.dump(env, f, indent=2)

print("=== SpectraMind V50 — Notebook 08 ===")
print(json.dumps(env, indent=2))


## Compose Hydra overrides for Ensembles, MC Dropout, Calibration & COREL

**Toggles (adjust to your repo’s `configs/`):**
- `uq.ensemble.enable=true`, `uq.ensemble.size=5`
- `uq.mc_dropout.enable=true`, with `keep_prob` or `p`, and `mc_samples=30`
- `calibration.temperature.enable=true` (optionally per-wavelength)
- `conformal.corel.enable=true`, e.g., `coverage=0.9`, graph spec for AIRS bins

We keep epochs small for demo; run ablations later for full sweeps.


In [ ]:
import json, os

overrides = {
    # Base runtime
    "data": "ariel_nominal",
    "model": "v50",                 # swap with 'hf_vit' or others if desired
    "training.max_epochs": "6",
    "training.batch_size": "16",
    "training.seed": "1337",
    # Ensemble
    "uq.ensemble.enable": "true",
    "uq.ensemble.size": "5",
    # MC Dropout
    "uq.mc_dropout.enable": "true",
    "uq.mc_dropout.p": "0.1",
    "uq.mc_dropout.mc_samples": "30",
    # Temperature scaling
    "calibration.temperature.enable": "true",
    "calibration.temperature.per_wavelength": "false",
    # COREL conformal
    "conformal.corel.enable": "true",
    "conformal.corel.coverage": "0.90",
    "conformal.corel.graph": "airs_default",
    # Mixed precision (optional)
    "training.mixed_precision": "fp16",
}

cfg_file = os.path.join(CFG_OUT, "uq_corel_overrides.json")
with open(cfg_file, "w") as f:
    json.dump(overrides, f, indent=2)
print("Saved overrides ->", cfg_file)
print(json.dumps(overrides, indent=2))


## Helper: robust CLI runner (DRY-RUN when CLI not present)

In [ ]:
import shlex, time

def run_cli(cmd_list, log_name="run"):
    log_path = os.path.join(LOGS, f"{log_name}.log")
    err_path = os.path.join(LOGS, f"{log_name}.err")
    start = time.time()
    result = {"cmd": cmd_list, "dry_run": DRY_RUN, "returncode": 0, "stdout": "", "stderr": ""}
    if DRY_RUN:
        msg = f"[DRY-RUN] Would execute: {' '.join(shlex.quote(c) for c in cmd_list)}\n"
        result["stdout"] = msg
        with open(log_path, "w") as f: f.write(msg)
        with open(err_path, "w") as f: f.write("")
        placeholder = os.path.join(ARTIFACTS, "dry_run_placeholder.txt")
        with open(placeholder, "a") as f: f.write(msg)
        return result

    with open(log_path, "wb") as out, open(err_path, "wb") as err:
        try:
            proc = subprocess.Popen(cmd_list, stdout=out, stderr=err, env=os.environ.copy())
            proc.wait()
            result["returncode"] = proc.returncode
        except Exception as e:
            result["returncode"] = 99
            with open(err_path, "ab") as errf:
                errf.write(str(e).encode())

    try:
        result["stdout"] = open(log_path, "r").read()
    except Exception:
        pass
    try:
        result["stderr"] = open(err_path, "r").read()
    except Exception:
        pass
    result["elapsed_sec"] = round(time.time() - start, 3)
    print(f"[rc={result['returncode']}] logs: {log_path}")
    return result


## Train **K** ensemble members

In [ ]:
K = int(overrides.get("uq.ensemble.size", "5"))
base_seed = int(overrides.get("training.seed", "1337"))
run_dirs = []

for k in range(K):
    seed = base_seed + k
    run_name = f"member_{k:02d}_seed_{seed}"
    out_dir = os.path.join(PRED_OUT, run_name)
    os.makedirs(out_dir, exist_ok=True)
    run_dirs.append(out_dir)

    cmd = ["spectramind", "train",
           "--config-name", "config_v50.yaml",
           "+outputs.root_dir=" + out_dir,
           f"+training.seed={seed}"]
    for key, val in overrides.items():
        cmd.append(f"+{key}={val}")
    cmd += ["+training.fast_mode=true"]
    res = run_cli(cmd, log_name=f"01_train_{run_name}")
    print(res["stdout"][:300])


## Predict for each member and **aggregate**

In [ ]:
pred_dirs = []
for out_dir in run_dirs:
    pred_dir = os.path.join(out_dir, "pred")
    os.makedirs(pred_dir, exist_ok=True)
    pred_dirs.append(pred_dir)
    cmd_pred = ["spectramind", "predict",
                "--config-name", "config_v50.yaml",
                "+load.from_checkpoint=true",
                "+outputs.root_dir=" + pred_dir]
    for key, val in overrides.items():
        cmd_pred.append(f"+{key}={val}")
    cmd_pred += ["+inference.save_mu_sigma=true"]
    res_p = run_cli(cmd_pred, log_name=f"02_predict_{os.path.basename(out_dir)}")
    print(res_p["stdout"][:300])

agg_dir = os.path.join(PRED_OUT, "ensemble_aggregate")
os.makedirs(agg_dir, exist_ok=True)
cmd_agg = ["spectramind", "diagnose", "ensemble-aggregate",
           "--inputs"] + pred_dirs + ["--out", agg_dir]
res_agg = run_cli(cmd_agg, log_name="03_ensemble_aggregate")
print(res_agg["stdout"][:300])


## Temperature scaling (post-hoc)

In [ ]:
cal_dir = os.path.join(DIAG_OUT, "temperature_scaling")
os.makedirs(cal_dir, exist_ok=True)
cmd_ts = ["spectramind", "calibrate", "temperature",
          "--pred", os.path.join(PRED_OUT, "ensemble_aggregate"),
          "--out", cal_dir,
          "--mode", "global"]
res_ts = run_cli(cmd_ts, log_name="04_temperature_scaling")
print(res_ts["stdout"][:300])


## COREL conformal calibration

In [ ]:
corel_dir = os.path.join(DIAG_OUT, "corel")
os.makedirs(corel_dir, exist_ok=True)
cmd_corel_fit = ["spectramind", "conformal", "corel-fit",
                 "--pred", os.path.join(PRED_OUT, "ensemble_aggregate"),
                 "--out", corel_dir,
                 "--coverage", overrides.get("conformal.corel.coverage", "0.90"),
                 "--graph", overrides.get("conformal.corel.graph", "airs_default")]
res_corel_fit = run_cli(cmd_corel_fit, log_name="05_corel_fit")
print(res_corel_fit["stdout"][:300])

cmd_corel_eval = ["spectramind", "conformal", "corel-eval",
                  "--model", corel_dir, "--out", os.path.join(corel_dir, "eval.json")]
res_corel_eval = run_cli(cmd_corel_eval, log_name="06_corel_eval")
print(res_corel_eval["stdout"][:300])


## Diagnostics dashboard

In [ ]:
dash_path = os.path.join(DIAG_OUT, "diagnostic_report_uq_corel_v1.html")
cmd_dash = ["spectramind", "diagnose", "dashboard",
            "--out", dash_path,
            "--include-coverage", "true"]
res_dash = run_cli(cmd_dash, log_name="07_dashboard")
print(res_dash["stdout"][:300])
print("Dashboard:", dash_path)


## Browse produced artifacts

In [ ]:
import os

def tree(path, prefix=""):
    items = sorted(os.listdir(path))
    lines = []
    for i, name in enumerate(items):
        full = os.path.join(path, name)
        connector = "└── " if i == len(items)-1 else "├── "
        lines.append(prefix + connector + name)
        if os.path.isdir(full):
            extension = "    " if i == len(items)-1 else "│   "
            lines.extend(tree(full, prefix + extension))
    return lines

print("ARTIFACTS TREE:", ARTIFACTS)
print("\n".join(tree(ARTIFACTS)))


## Pipeline sketch (Mermaid)

```mermaid
flowchart LR
  A[Train K ensemble members] --> B[Predict each + MC Dropout]
  B --> C[Aggregate μ,σ; compute epistemic variance]
  C --> D[Temp scaling (post-hoc)]
  D --> E[COREL conformal fit + eval (coverage)]
  E --> F[Diagnostics dashboard / CI]
```

## Next steps
- Sweep **ensemble size, MC samples, dropout p**, and **coverage** via `spectramind ablate`.
- Try **per-wavelength** temperature scaling (if supported) and compare GLL/coverage.
- Export JSON summaries for CI gating and archive HTML in versioned reports.


## 09_experiment_tracking_dvc_ci.ipynb

# 🧪 SpectraMind V50 — Experiment Tracking, DVC DAG & CI (Notebook 09)

**Purpose.** Close the reproducibility loop by wiring **experiment tracking**, **DVC pipeline/DAG**, and **CI artifacts**:
- Track runs (config → metrics → artifacts) with **MLflow** (or structured JSON fallback).
- Define/validate a **DVC DAG** for calibration → train → predict → diagnostics stages.
- Emit CI-friendly artifacts (JSON summaries, HTML reports) and run **self-test** hooks.

This continues the sequence after 08 (ensembles/MC Dropout/COREL) and focuses on **engineering rigor**.


In [ ]:
# ░░ Pre-flight: env, run IDs, paths, CLI detection ░░
import os, sys, json, platform, shutil, subprocess, datetime, pathlib

RUN_TS = datetime.datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
RUN_ID = f"tracking_dvc_ci_{RUN_TS}"
ROOT_OUT = "/mnt/data/tracking_dvc_ci"
ARTIFACTS = os.path.join(ROOT_OUT, RUN_ID)
LOGS = os.path.join(ARTIFACTS, "logs")
CFG_OUT = os.path.join(ARTIFACTS, "configs")
CI_OUT = os.path.join(ARTIFACTS, "ci_artifacts")
MLF_OUT = os.path.join(ARTIFACTS, "mlruns")  # default local store if mlflow used
for p in (ROOT_OUT, ARTIFACTS, LOGS, CFG_OUT, CI_OUT, MLF_OUT):
    os.makedirs(p, exist_ok=True)

def which(cmd: str) -> bool:
    return shutil.which(cmd) is not None

CLI_PRESENT = which("spectramind")
DRY_RUN = not CLI_PRESENT

def git_cmd(args):
    try:
        out = subprocess.check_output(["git", *args], stderr=subprocess.STDOUT, timeout=5).decode().strip()
        return out
    except Exception:
        return None

env = {
    "python": sys.version.replace("\n", " "),
    "platform": platform.platform(),
    "cli_present": CLI_PRESENT,
    "dry_run": DRY_RUN,
    "run_id": RUN_ID,
    "paths": {"artifacts": ARTIFACTS, "logs": LOGS, "configs": CFG_OUT, "ci": CI_OUT, "mlruns": MLF_OUT},
    "git": {
        "commit": git_cmd(["rev-parse", "HEAD"]),
        "branch": git_cmd(["rev-parse", "--abbrev-ref", "HEAD"]),
        "status": git_cmd(["status", "--porcelain"]),
    },
}
with open(os.path.join(ARTIFACTS, "env.json"), "w") as f:
    json.dump(env, f, indent=2)

print("=== SpectraMind V50 — Notebook 09 ===")
print(json.dumps(env, indent=2))


## Experiment Tracking: MLflow (with structured JSON fallback)

We attempt to use **MLflow**; if unavailable, we record runs in `events.jsonl` + `summary.json` so CI can ingest metrics.


In [ ]:
MLFLOW_OK = False
try:
    import mlflow  # type: ignore
    MLFLOW_OK = True
except Exception:
    MLFLOW_OK = False

print("MLflow available:", MLFLOW_OK)
EVENTS = os.path.join(ARTIFACTS, "events.jsonl")
SUMMARY = os.path.join(ARTIFACTS, "summary.json")


## Compose Hydra overrides for a small demo run

We keep it light here; use your ablation tooling for full sweeps. The goal is to demonstrate **tracked runs**.


In [ ]:
import json, os

overrides = {
    "data": "ariel_nominal",
    "model": "v50",
    "training.max_epochs": "3",
    "training.batch_size": "16",
    "training.seed": "2025",
    # Diagnostics toggles for predictable outputs
    "diagnostics.export_json": "true",
}
cfg_file = os.path.join(CFG_OUT, "tracking_overrides.json")
with open(cfg_file, "w") as f:
    json.dump(overrides, f, indent=2)
print("Saved overrides ->", cfg_file)
print(json.dumps(overrides, indent=2))


## Helper: robust CLI runner (DRY-RUN if CLI not present)

In [ ]:
import shlex, time

def run_cli(cmd_list, log_name="run"):
    log_path = os.path.join(LOGS, f"{log_name}.log")
    err_path = os.path.join(LOGS, f"{log_name}.err")
    start = time.time()
    result = {"cmd": cmd_list, "dry_run": DRY_RUN, "returncode": 0, "stdout": "", "stderr": ""}
    if DRY_RUN:
        msg = f"[DRY-RUN] Would execute: {' '.join(shlex.quote(c) for c in cmd_list)}\n"
        result["stdout"] = msg
        with open(log_path, "w") as f: f.write(msg)
        with open(err_path, "w") as f: f.write("")
        with open(os.path.join(ARTIFACTS, "events.jsonl"), "a") as ev:
            ev.write(json.dumps({"ts": RUN_TS, "event": "dry_run", "cmd": result["cmd"]}) + "\n")
        return result

    with open(log_path, "wb") as out, open(err_path, "wb") as err:
        try:
            proc = subprocess.Popen(cmd_list, stdout=out, stderr=err, env=os.environ.copy())
            proc.wait()
            result["returncode"] = proc.returncode
        except Exception as e:
            result["returncode"] = 99
            with open(err_path, "ab") as errf:
                errf.write(str(e).encode())

    try: result["stdout"] = open(log_path, "r").read()
    except Exception: pass
    try: result["stderr"] = open(err_path, "r").read()
    except Exception: pass
    result["elapsed_sec"] = round(time.time() - start, 3)
    print(f"[rc={result['returncode']}] logs: {log_path}")
    return result


## Tracked demo run: `train → predict → diagnose`

- If MLflow is present, we start a run, log params/metrics/artifacts.
- Otherwise, we append to `events.jsonl` and assemble `summary.json` for CI.


In [ ]:
import uuid, time, json, os, glob

run_uid = str(uuid.uuid4())

def log_fallback(record: dict):
    with open(EVENTS, "a") as ev:
        ev.write(json.dumps(record) + "\n")

# Start run
if MLFLOW_OK:
    mlflow.set_tracking_uri("file://" + os.path.abspath(MLF_OUT))
    mlflow.set_experiment("spectramind_v50")
    mlflow.start_run(run_name=f"demo_{run_uid}")
    mlflow.log_params(overrides)
else:
    log_fallback({"event": "start_run", "run_id": run_uid, "params": overrides, "ts": time.time()})

# train
cmd_train = ["spectramind", "train", "--config-name", "config_v50.yaml", "+outputs.root_dir=" + ARTIFACTS]
for k, v in overrides.items():
    cmd_train.append(f"+{k}={v}")
res_tr = run_cli(cmd_train, log_name="01_train")
if MLFLOW_OK:
    mlflow.log_text(res_tr["stdout"][:1000], "logs/train_head.txt")
else:
    log_fallback({"event": "train_done", "rc": res_tr["returncode"], "ts": time.time()})

# predict
pred_dir = os.path.join(ARTIFACTS, "pred")
os.makedirs(pred_dir, exist_ok=True)
cmd_pred = ["spectramind", "predict", "--config-name", "config_v50.yaml", "+load.from_checkpoint=true", "+outputs.root_dir=" + pred_dir]
for k, v in overrides.items():
    cmd_pred.append(f"+{k}={v}")
cmd_pred += ["+inference.save_mu_sigma=true"]
res_pr = run_cli(cmd_pred, log_name="02_predict")
if MLFLOW_OK:
    mlflow.log_text(res_pr["stdout"][:1000], "logs/predict_head.txt")
else:
    log_fallback({"event": "predict_done", "rc": res_pr["returncode"], "ts": time.time()})

# diagnose dashboard
diag_dir = os.path.join(ARTIFACTS, "diagnostics")
os.makedirs(diag_dir, exist_ok=True)
dashboard = os.path.join(diag_dir, "diagnostic_report_v1.html")
cmd_diag = ["spectramind", "diagnose", "dashboard", "--out", dashboard]
res_dg = run_cli(cmd_diag, log_name="03_diagnose")
if MLFLOW_OK:
    if os.path.exists(dashboard):
        mlflow.log_artifact(dashboard, artifact_path="reports")
    mlflow.log_text(res_dg["stdout"][:1000], "logs/diagnose_head.txt")
else:
    log_fallback({"event": "diagnose_done", "rc": res_dg["returncode"], "dashboard": dashboard, "ts": time.time()})

# Finish run
if MLFLOW_OK:
    # Example metric emit (replace with real metrics from diagnostics JSON if available)
    mlflow.log_metric("demo_metric_gll", 0.0)
    mlflow.end_run()
else:
    with open(SUMMARY, "w") as f:
        json.dump({"run_id": run_uid, "metrics": {"demo_metric_gll": 0.0}}, f, indent=2)
print("Tracking complete.")


## DVC DAG: emit stage templates

We write **DVC stage YAML fragments** for `calibrate → train → predict → diagnose` so you can paste or integrate into `dvc.yaml`.


In [ ]:
dvc_frag = os.path.join(ARTIFACTS, "dvc_stages.yaml")
frag = f"""
stages:
  calibrate:
    cmd: spectramind calibrate --config-name config_v50.yaml
    deps:
    - configs
    outs:
    - data/calibrated
  train:
    cmd: spectramind train --config-name config_v50.yaml
    deps:
    - data/calibrated
    - configs
    outs:
    - models/v50_checkpoint
  predict:
    cmd: spectramind predict --config-name config_v50.yaml +load.from_checkpoint=true
    deps:
    - models/v50_checkpoint
    outs:
    - outputs/predictions
  diagnose:
    cmd: spectramind diagnose dashboard --out outputs/diagnostic_report_v1.html
    deps:
    - outputs/predictions
    outs:
    - outputs/diagnostic_report_v1.html
"""
open(dvc_frag, "w").write(frag)
print("Wrote DVC stage template ->", dvc_frag)
print(open(dvc_frag).read())


## CI artifacts: package JSON summaries & HTML dashboard

We copy key files to the `ci_artifacts/` folder so a CI job can always upload them.


In [ ]:
import shutil, os, glob

# Collect likely artifacts if present; tolerate absence in DRY-RUN
maybe = []
for path in [
    os.path.join(ARTIFACTS, "diagnostics", "diagnostic_report_v1.html"),
    os.path.join(ARTIFACTS, "summary.json"),
    os.path.join(ARTIFACTS, "events.jsonl"),
]:
    if os.path.exists(path):
        maybe.append(path)

for pth in maybe:
    shutil.copy2(pth, CI_OUT)

print("CI bundle contains:", os.listdir(CI_OUT))


## Browse produced artifacts

In [ ]:
import os

def tree(path, prefix=""):
    items = sorted(os.listdir(path))
    lines = []
    for i, name in enumerate(items):
        full = os.path.join(path, name)
        connector = "└── " if i == len(items)-1 else "├── "
        lines.append(prefix + connector + name)
        if os.path.isdir(full):
            extension = "    " if i == len(items)-1 else "│   "
            lines.extend(tree(full, prefix + extension))
    return lines

print("ARTIFACTS TREE:", ARTIFACTS)
print("\n".join(tree(ARTIFACTS)))
print("CI artifacts:", os.listdir(os.path.join(ARTIFACTS, "ci_artifacts")))


## Pipeline sketch (Mermaid)

```mermaid
flowchart LR
  A[Hydra config] --> B[Calibrate]
  B --> C[Train (tracked)]
  C --> D[Predict]
  D --> E[Diagnose → HTML]
  C --> F[Metrics JSON/MLflow]
  E --> G[CI Upload]
  F --> G
```

## Next steps
- Wire **real metrics** from diagnostics JSON into MLflow/summary outputs.
- Paste `dvc_stages.yaml` content into `dvc.yaml` and `dvc repro` to validate the DAG on small data.
- Hook this notebook (or the CLI) into **CI** to auto-run `selftest` + produce a **CI bundle** on every PR.


---
# 🔗 Additional Stitched Modules
Imported after your latest upload.

## 10_full_pipeline_reproducibility_and_ci.ipynb

# 🔁 SpectraMind V50 — Full Pipeline Reproducibility & CI (Notebook 10)

**Goal.** Execute the **entire pipeline end‑to‑end** in a controlled, CLI‑first flow; capture **exact config + data + code provenance**; and emit a **reproducibility manifest** compatible with CI.

**What this notebook does**
1. Pre‑flight & environment capture (CLI/DVC detection, git info, run ID)
2. DVC quick status (cache/stage checks) with graceful fallback
3. End‑to‑end pipeline run (calibrate → train → diagnose → submit) with **DRY‑RUN fallback** if `spectramind` is not available
4. Create a **reproducibility manifest** (commit hash, config snapshot if available, artifact hashes)
5. CI smoke‑test hooks (idempotent rerun check; basic integrity assertions)
6. Artifact tree + next steps

> As in earlier notebooks, everything is **CLI‑first, Hydra‑safe**, and will **degrade gracefully** if local tools are not present.


In [ ]:
# ░░ Pre-flight: detect tools, set paths, capture git/env ░░
import os, sys, json, shutil, subprocess, datetime, pathlib

RUN_TS = datetime.datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
RUN_ID = f"full_pipeline_ci_{RUN_TS}"
ROOT_OUT = "/mnt/data/full_pipeline_ci"
ARTIFACTS = os.path.join(ROOT_OUT, RUN_ID)
LOGS = os.path.join(ARTIFACTS, "logs")
CFG_OUT = os.path.join(ARTIFACTS, "configs")
DIAG_OUT = os.path.join(ARTIFACTS, "diagnostics")
SUBMIT_OUT = os.path.join(ARTIFACTS, "submission")
for p in (ROOT_OUT, ARTIFACTS, LOGS, CFG_OUT, DIAG_OUT, SUBMIT_OUT):
    os.makedirs(p, exist_ok=True)

def which(cmd:str)->bool: return shutil.which(cmd) is not None
CLI_PRESENT = which("spectramind")
DVC_PRESENT = which("dvc")

def git_cmd(args):
    try:
        out = subprocess.check_output(["git", *args], stderr=subprocess.STDOUT, timeout=5).decode().strip()
        return out
    except Exception:
        return None

env = {
    "python": sys.version.replace("\n"," "),
    "platform": sys.platform,
    "cli_present": CLI_PRESENT,
    "dvc_present": DVC_PRESENT,
    "run_id": RUN_ID,
    "paths": {"artifacts": ARTIFACTS, "logs": LOGS, "configs": CFG_OUT, "diagnostics": DIAG_OUT, "submission": SUBMIT_OUT},
    "git": {
        "commit": git_cmd(["rev-parse", "HEAD"]),
        "branch": git_cmd(["rev-parse", "--abbrev-ref", "HEAD"]),
        "status": git_cmd(["status", "--porcelain"]),
        "remote": git_cmd(["remote", "-v"]),
    },
}
with open(os.path.join(ARTIFACTS, "env.json"), "w") as f:
    json.dump(env, f, indent=2)

print("=== Pre-flight ===")
print(json.dumps(env, indent=2))


## DVC quick status (graceful fallback)

In [ ]:
import subprocess, pathlib, json, os

dvc_info = {"present": DVC_PRESENT, "version": None, "status": None}
if DVC_PRESENT:
    try:
        ver = subprocess.check_output(["dvc", "--version"], timeout=10).decode().strip()
        dvc_info["version"] = ver
    except Exception as e:
        dvc_info["version"] = f"error: {e}"
    try:
        # Check for cached stage status; if no DVC repo, this will fail
        out = subprocess.check_output(["dvc", "status", "-c"], stderr=subprocess.STDOUT, timeout=15).decode()
        dvc_info["status"] = out
    except Exception as e:
        dvc_info["status"] = f"(no DVC repo or error) {e}"
else:
    dvc_info["status"] = "(dvc not installed)"
with open(os.path.join(ARTIFACTS, "dvc_status.json"), "w") as f:
    json.dump(dvc_info, f, indent=2)
print(json.dumps(dvc_info, indent=2))


## Helper: robust CLI runner (DRY‑RUN when CLI missing)

In [ ]:
import shlex, time

def run_cli(cmd_list, log_name="run"):
    log_path = os.path.join(LOGS, f"{log_name}.log")
    err_path = os.path.join(LOGS, f"{log_name}.err")
    start = time.time()
    result = {"cmd": cmd_list, "dry_run": not CLI_PRESENT, "returncode": 0, "stdout": "", "stderr": ""}
    if not CLI_PRESENT:
        msg = f"[DRY-RUN] Would execute: {' '.join(shlex.quote(c) for c in cmd_list)}\n"
        result["stdout"] = msg
        with open(log_path, "w") as f: f.write(msg)
        with open(err_path, "w") as f: f.write("")
        return result

    with open(log_path, "wb") as out, open(err_path, "wb") as err:
        try:
            proc = subprocess.Popen(cmd_list, stdout=out, stderr=err, env=os.environ.copy())
            proc.wait()
            result["returncode"] = proc.returncode
        except Exception as e:
            result["returncode"] = 99
            with open(err_path, "ab") as errf:
                errf.write(str(e).encode())

    try: result["stdout"] = open(log_path, "r").read()
    except Exception: pass
    try: result["stderr"] = open(err_path, "r").read()
    except Exception: pass
    result["elapsed_sec"] = round(time.time() - start, 3)
    print(f"[rc={result['returncode']}] logs: {log_path}")
    return result


## End‑to‑end pipeline run

In [ ]:
# Calibrate (optionally subset for a fast CI smoke run)
cal_cmd = [
    "spectramind","calibrate",
    "+outputs.root_dir="+ARTIFACTS,
    # Optional flags if supported by your CLI:
    # "--sample","8"
]
res_cal = run_cli(cal_cmd, log_name="01_calibrate")
print(res_cal["stdout"][:400])

# Train (fast mode for CI; bump epochs in full runs)
train_cmd = [
    "spectramind","train",
    "--config-name","config_v50.yaml",
    "+outputs.root_dir="+ARTIFACTS,
    "+training.max_epochs=1",
    "+training.fast_mode=true"
]
res_tr = run_cli(train_cmd, log_name="02_train")
print(res_tr["stdout"][:400])

# Diagnose (HTML dashboard)
dash_html = os.path.join(DIAG_OUT, "diagnostic_report_ci_v1.html")
diag_cmd = ["spectramind","diagnose","dashboard","--out", dash_html]
res_dg = run_cli(diag_cmd, log_name="03_diagnose_dashboard")
print(res_dg["stdout"][:400])

# Submit bundle (pack outputs)
submit_zip = os.path.join(SUBMIT_OUT, "submission_bundle.zip")
sub_cmd = ["spectramind","submit","--out", submit_zip]
res_sb = run_cli(sub_cmd, log_name="04_submit")
print(res_sb["stdout"][:400])


## Build a reproducibility manifest

In [ ]:
import hashlib, json, glob, os

def sha256_of_file(path, chunk=1024*1024):
    try:
        h = hashlib.sha256()
        with open(path, "rb") as f:
            while True:
                b = f.read(chunk)
                if not b: break
                h.update(b)
        return h.hexdigest()
    except Exception:
        return None

# Collect artifacts and hashes
artifact_files = []
for root, dirs, files in os.walk(ARTIFACTS):
    for fn in files:
        full = os.path.join(root, fn)
        rel = os.path.relpath(full, ARTIFACTS)
        artifact_files.append({"path": rel, "sha256": sha256_of_file(full)})

manifest = {
    "run_id": RUN_ID,
    "timestamp_utc": RUN_TS,
    "git": env.get("git"),
    "cli_present": CLI_PRESENT,
    "dvc": json.load(open(os.path.join(ARTIFACTS, "dvc_status.json"))),
    "commands": {
        "calibrate": res_cal["cmd"],
        "train": res_tr["cmd"],
        "diagnose_dashboard": res_dg["cmd"],
        "submit": res_sb["cmd"],
    },
    "returncodes": {
        "calibrate": res_cal["returncode"],
        "train": res_tr["returncode"],
        "diagnose_dashboard": res_dg["returncode"],
        "submit": res_sb["returncode"],
    },
    "artifacts": artifact_files,
    "notes": "Config snapshots will be included if the CLI dumps composed Hydra configs into outputs. DRY-RUN indicates missing CLI."
}
with open(os.path.join(ARTIFACTS, "repro_manifest.json"), "w") as f:
    json.dump(manifest, f, indent=2)

print(f"Manifest written to: {os.path.join(ARTIFACTS, 'repro_manifest.json')}")
print(f"Artifacts counted: {len(artifact_files)}")


## CI smoke‑test checks

In [ ]:
# Simple checks that help CI verify integrity
import json, os

rc_ok = all(x == 0 for x in [
    res_cal["returncode"],
    res_tr["returncode"],
    res_dg["returncode"],
    res_sb["returncode"],
]) if CLI_PRESENT else True  # In DRY-RUN, allow pass

manifest_path = os.path.join(ARTIFACTS, "repro_manifest.json")
has_manifest = os.path.exists(manifest_path)

print("Return codes OK (or DRY-RUN):", rc_ok)
print("Manifest exists:", has_manifest)

# (Optional) Idempotent rerun of a tiny step, e.g., diagnose-only
# This is skipped in DRY-RUN to keep the notebook fast & deterministic.
if CLI_PRESENT:
    res_dg2 = run_cli(["spectramind","diagnose","dashboard","--out", os.path.join(DIAG_OUT,"diagnostic_report_ci_v2.html")], log_name="05_diagnose_dashboard_rerun")
    print("Second dashboard rc:", res_dg2["returncode"])
else:
    print("[DRY-RUN] Skipping idempotent re-run.")

assert has_manifest, "Reproducibility manifest missing."
print("CI smoke-test checks complete.")


## Browse produced artifacts

In [ ]:
import os

def tree(path, prefix=""):
    items = sorted(os.listdir(path))
    lines = []
    for i, name in enumerate(items):
        full = os.path.join(path, name)
        connector = "└── " if i == len(items)-1 else "├── "
        lines.append(prefix + connector + name)
        if os.path.isdir(full):
            extension = "    " if i == len(items)-1 else "│   "
            lines.extend(tree(full, prefix + extension))
    return lines

print("ARTIFACTS TREE:", ARTIFACTS)
print("\n".join(tree(ARTIFACTS)))


## Pipeline sketch (Mermaid)

```mermaid
flowchart LR
  A[Calibrate] --> B[Train]
  B --> C[Diagnose → HTML]
  C --> D[Submit → ZIP]
  A -.->|DVC stages| B
  B -.->|Hydra config| C
  C -.->|Manifest hashes| D
```


## Next steps
- Commit the generated `repro_manifest.json` and artifacts (or track large ones via **DVC**).
- Add a **CI job** (GitHub Actions) that runs this notebook or its equivalent CLI sequence on a schedule and on PRs.
- Ensure the **CLI dumps composed Hydra configs** into `configs/` per run (the manifest will capture them automatically).
- Consider adding **MLflow** run logging alongside DVC for a web UI comparison of runs.

If you want, I can also prepare a **GitHub Actions** CI YAML that runs the same smoke‑test using the CLI only.


## 11_kaggle_submission_and_leaderboard_playbook.ipynb

# 🏁 SpectraMind V50 — Kaggle Submission & Leaderboard Playbook (Notebook 11)

**Goal.** Package, validate, and (optionally) upload a **Kaggle submission** for the NeurIPS Ariel Data Challenge.  
This notebook is **CLI-first** and provides **DRY-RUN** safety if the `kaggle` CLI isn't available.

**What this notebook does**
1. Pre-flight: detect Kaggle CLI, capture env/git info, set run paths  
2. Locate/validate the submission artifacts (CSV/ZIP) produced by `spectramind submit`  
3. Create a **submission bundle** + README/model card and a **manifest** with hashes  
4. (Optional) **Kaggle upload** via CLI/API — with **DRY-RUN fallback**  
5. Record **leaderboard metadata** and a submission log usable in CI and postmortems  
6. Mermaid sketch of the submission workflow


In [ ]:
# ░░ Pre-flight ░░
import os, sys, json, shutil, subprocess, datetime, pathlib, hashlib

RUN_TS = datetime.datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
RUN_ID = f"kaggle_submit_{RUN_TS}"
ROOT_OUT = "/mnt/data/kaggle_submission"
ARTIFACTS = os.path.join(ROOT_OUT, RUN_ID)
LOGS = os.path.join(ARTIFACTS, "logs")
PKG = os.path.join(ARTIFACTS, "package")
for p in (ROOT_OUT, ARTIFACTS, LOGS, PKG):
    os.makedirs(p, exist_ok=True)

def which(cmd:str)->bool: return shutil.which(cmd) is not None
KAGGLE_PRESENT = which("kaggle")
CLI_PRESENT = which("spectramind")

def git_cmd(args):
    try:
        out = subprocess.check_output(["git", *args], stderr=subprocess.STDOUT, timeout=5).decode().strip()
        return out
    except Exception:
        return None

env = {
    "python": sys.version.replace("\n"," "),
    "platform": sys.platform,
    "kaggle_present": KAGGLE_PRESENT,
    "spectramind_present": CLI_PRESENT,
    "run_id": RUN_ID,
    "paths": {"artifacts": ARTIFACTS, "logs": LOGS, "package": PKG},
    "git": {
        "commit": git_cmd(["rev-parse", "HEAD"]),
        "branch": git_cmd(["rev-parse", "--abbrev-ref", "HEAD"]),
        "status": git_cmd(["status", "--porcelain"]),
    },
}
with open(os.path.join(ARTIFACTS, "env.json"), "w") as f:
    json.dump(env, f, indent=2)

print("=== Pre-flight ===")
print(json.dumps(env, indent=2))


## Locate submission artifact (CSV/ZIP)

In [ ]:
import glob, os, json, pathlib

# Heuristics: look for a submission file produced earlier (e.g., by `spectramind submit`)
candidate_globs = [
    "/mnt/data/**/submission*.csv",
    "/mnt/data/**/submission*.zip",
    "/mnt/data/**/submission_bundle*.zip",
]
found = []
for pattern in candidate_globs:
    for path in glob.glob(pattern, recursive=True):
        if os.path.isfile(path):
            found.append(path)

found = sorted(set(found), key=lambda p: (os.path.getmtime(p), p), reverse=True)
print("Found candidate submissions:", json.dumps(found[:10], indent=2))

SUBMISSION_FILE = found[0] if found else None
print("Chosen submission file:", SUBMISSION_FILE)


## Validate & stage submission

In [ ]:
import shutil, os, csv, zipfile, json, hashlib

def sha256_of_file(path, chunk=1024*1024):
    try:
        h = hashlib.sha256()
        with open(path, "rb") as f:
            while True:
                b = f.read(chunk)
                if not b: break
                h.update(b)
        return h.hexdigest()
    except Exception:
        return None

valid = False
msg = ""

if SUBMISSION_FILE and os.path.isfile(SUBMISSION_FILE):
    # Basic checks: csv or zip
    ext = os.path.splitext(SUBMISSION_FILE)[1].lower()
    if ext == ".csv":
        # Minimal CSV sanity: header present, at least one row
        try:
            with open(SUBMISSION_FILE, newline="") as f:
                reader = csv.reader(f)
                header = next(reader, None)
                row = next(reader, None)
                valid = header is not None and row is not None
                msg = f"CSV header={header} first_row={row[:3] if row else None}"
        except Exception as e:
            msg = f"CSV read error: {e}"
    elif ext == ".zip":
        try:
            with zipfile.ZipFile(SUBMISSION_FILE, "r") as zf:
                namelist = zf.namelist()
                valid = len(namelist) > 0
                msg = f"ZIP contains: {namelist[:5]}..."
        except Exception as e:
            msg = f"ZIP read error: {e}"
    else:
        msg = f"Unsupported extension: {ext}"
else:
    msg = "No submission file found."

print("Valid?", valid, "|", msg)

STAGED = None
if valid:
    STAGED = os.path.join(PKG, os.path.basename(SUBMISSION_FILE))
    shutil.copy2(SUBMISSION_FILE, STAGED)
    print("Staged:", STAGED, "SHA256:", sha256_of_file(STAGED))
else:
    print("Skipping stage; invalid or missing submission file.")


## Write README/model card & manifest

In [ ]:
readme = f"""# SpectraMind V50 — Kaggle Submission Package

**Run ID:** {RUN_ID}  
**Timestamp (UTC):** {RUN_TS}

This package was generated by *Notebook 11 — Kaggle Submission & Leaderboard Playbook*.

## Contents
- `{os.path.basename(STAGED) if STAGED else 'MISSING'}` — submission artifact
- `manifest.json` — provenance (git, hashes)
- `notes.md` — optional notes

## Reproducibility
- Code commit: {env['git']['commit']}
- Branch: {env['git']['branch']}
- Python: {env['python']}

This package is designed to be CI-friendly and traceable.
"""

with open(os.path.join(PKG, "README.md"), "w") as f:
    f.write(readme)

manifest = {
    "run_id": RUN_ID,
    "timestamp_utc": RUN_TS,
    "git": env.get("git"),
    "submission_file": STAGED,
    "submission_sha256": sha256_of_file(STAGED) if STAGED else None,
    "kaggle_cli_present": KAGGLE_PRESENT,
}
with open(os.path.join(PKG, "manifest.json"), "w") as f:
    json.dump(manifest, f, indent=2)

with open(os.path.join(PKG, "notes.md"), "w") as f:
    f.write("Add experiment notes or leaderboard observations here.\n")
    
print("Wrote README, manifest, notes into", PKG)


## (Optional) Upload to Kaggle — DRY-RUN safe

In [ ]:
import subprocess, shlex, os, json

def run_cmd(cmd_list, log_name):
    log_path = os.path.join(LOGS, f"{log_name}.log")
    err_path = os.path.join(LOGS, f"{log_name}.err")
    if not KAGGLE_PRESENT:
        msg = f"[DRY-RUN] Would execute: {' '.join(shlex.quote(c) for c in cmd_list)}\n"
        open(log_path, "w").write(msg); open(err_path, "w").write("")
        return 0, msg, ""
    try:
        proc = subprocess.Popen(cmd_list, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        out, err = proc.communicate()
        open(log_path, "wb").write(out or b""); open(err_path, "wb").write(err or b"")
        return proc.returncode, (out or b"").decode(), (err or b"").decode()
    except Exception as e:
        return 99, "", str(e)

# NOTE: adjust competition slug if needed
COMPETITION = "ariel-data-challenge-2025"

rc, out, err = (0, "", "")
if STAGED and os.path.isfile(STAGED):
    # Kaggle expects: kaggle competitions submit -c <comp> -f <file> -m "<message>"
    msg = f"SpectraMind V50 auto-submit {RUN_ID}"
    cmd = ["kaggle", "competitions", "submit", "-c", COMPETITION, "-f", STAGED, "-m", msg]
    rc, out, err = run_cmd(cmd, log_name="kaggle_submit")
    print("Submit rc:", rc)
    print("stdout (truncated):", out[:300])
    print("stderr (truncated):", err[:300])
else:
    print("[Skip] No staged submission file to upload.")


## Record submission log & leaderboard metadata stub

In [ ]:
log = {
    "run_id": RUN_ID,
    "ts_utc": RUN_TS,
    "kaggle_present": KAGGLE_PRESENT,
    "submitted_file": os.path.basename(STAGED) if STAGED else None,
    "submit_rc": rc if 'rc' in locals() else None,
}
with open(os.path.join(ARTIFACTS, "submission_log.json"), "w") as f:
    json.dump(log, f, indent=2)
print("Saved submission log:", os.path.join(ARTIFACTS, "submission_log.json"))


## Browse produced artifacts

In [ ]:
import os

def tree(path, prefix=""):
    items = sorted(os.listdir(path))
    lines = []
    for i, name in enumerate(items):
        full = os.path.join(path, name)
        connector = "└── " if i == len(items)-1 else "├── "
        lines.append(prefix + connector + name)
        if os.path.isdir(full):
            extension = "    " if i == len(items)-1 else "│   "
            lines.extend(tree(full, prefix + extension))
    return lines

print("PKG TREE:", PKG)
print("\n".join(tree(PKG)))


## Submission flow (Mermaid)

```mermaid
flowchart LR
  A[Find submission CSV/ZIP] --> B[Validate structure]
  B --> C[Stage into /package]
  C --> D[Write README + manifest]
  D --> E{Kaggle CLI available?}
  E -- Yes --> F[Upload via kaggle competitions submit]
  E -- No --> G[DRY-RUN: log command]
  F --> H[Submission log + LB notes]
  G --> H
```


## Next steps
- Ensure your **Kaggle API token** is configured (`~/.kaggle/kaggle.json`) with proper permissions.
- Verify the **competition slug** (default: `ariel-data-challenge-2025`) before uploading.
- Use this notebook in **CI** after `10_full_pipeline_reproducibility_and_ci.ipynb` to automate packaging and submission.
- Track submissions & scores in a lightweight CSV or use MLflow/Sheets for team visibility.

> Tip: read Kaggle’s platform guide for notebook/CLI usage and submission rules.


## 12_post_submission_analysis_and_leaderboard_tracking.ipynb

# 📊 SpectraMind V50 — Post-Submission Analysis & Leaderboard Tracking (Notebook 12)

**Goal.** Centralize post-submission metadata, track public leaderboard results, and maintain a lightweight, CI-friendly log of submissions and scores.

**What this notebook does**
1. Pre-flight (detect Kaggle CLI, set paths, read repo/git info)
2. Gather **local submission logs** (from prior notebooks) and consolidate into a historical CSV/JSON
3. (Optional) **Query Kaggle submissions** via CLI/API — DRY-RUN safe
4. Merge local logs with Kaggle metadata; compute **deltas** between runs and annotate best scoring submissions
5. Emit artifacts: `submissions_history.csv`, `best_submission.json`, and a **Mermaid** trend sketch


In [ ]:
# ░░ Pre-flight ░░
import os, sys, json, shutil, subprocess, datetime, pathlib, csv

RUN_TS = datetime.datetime.utcnow().strftime("%Y%m%dT%H%M%SZ")
RUN_ID = f"post_submit_{RUN_TS}"
ROOT_OUT = "/mnt/data/leaderboard_tracking"
ARTIFACTS = os.path.join(ROOT_OUT, RUN_ID)
LOGS = os.path.join(ARTIFACTS, "logs")
os.makedirs(ARTIFACTS, exist_ok=True); os.makedirs(LOGS, exist_ok=True)

def which(cmd:str)->bool: return shutil.which(cmd) is not None
KAGGLE_PRESENT = which("kaggle")

def git_cmd(args):
    try:
        out = subprocess.check_output(["git", *args], stderr=subprocess.STDOUT, timeout=5).decode().strip()
        return out
    except Exception:
        return None

env = {
    "python": sys.version.replace("\n"," "),
    "platform": sys.platform,
    "kaggle_present": KAGGLE_PRESENT,
    "run_id": RUN_ID,
    "paths": {"artifacts": ARTIFACTS, "logs": LOGS},
    "git": {
        "commit": git_cmd(["rev-parse", "HEAD"]),
        "branch": git_cmd(["rev-parse", "--abbrev-ref", "HEAD"]),
        "status": git_cmd(["status", "--porcelain"]),
    },
}
with open(os.path.join(ARTIFACTS, "env.json"), "w") as f:
    json.dump(env, f, indent=2)

print("=== Pre-flight ===")
print(json.dumps(env, indent=2))


## Consolidate local submission logs

In [ ]:
import glob, os, json, csv, hashlib

# Find prior local logs (from Notebook 11 or pipeline)
CANDIDATES = sorted(set(glob.glob("/mnt/data/**/submission_log.json", recursive=True)))
print("Found local logs:", len(CANDIDATES))

records = []
for path in CANDIDATES:
    try:
        data = json.load(open(path))
        # Normalize fields
        rec = {
            "source_path": path,
            "ts_utc": data.get("ts_utc") or data.get("timestamp_utc"),
            "run_id": data.get("run_id"),
            "submitted_file": data.get("submitted_file"),
            "kaggle_present": data.get("kaggle_present"),
            "submit_rc": data.get("submit_rc"),
        }
        # Attach file hash if available
        if rec["submitted_file"]:
            # try to resolve absolute path
            abs_guess = os.path.join(os.path.dirname(path), "..", "package", rec["submitted_file"])
            abs_guess = os.path.abspath(abs_guess)
            sha256 = None
            if os.path.exists(abs_guess):
                h = hashlib.sha256(); 
                with open(abs_guess, "rb") as f:
                    h.update(f.read())
                sha256 = h.hexdigest()
            rec["file_sha256"] = sha256
        records.append(rec)
    except Exception as e:
        print("Skip unreadable log:", path, e)

# Write a consolidated CSV snapshot for this run
snapshot_csv = os.path.join(ARTIFACTS, "submissions_snapshot.csv")
with open(snapshot_csv, "w", newline="") as f:
    w = csv.DictWriter(f, fieldnames=sorted({k for r in records for k in r.keys()}))
    w.writeheader()
    for r in records: w.writerow(r)

print("Snapshot rows:", len(records), "->", snapshot_csv)


## (Optional) Pull live Kaggle submission metadata — DRY-RUN safe

In [ ]:
import subprocess, json, os, shlex

COMPETITION = "ariel-data-challenge-2025"  # adjust if needed
def run_cmd(cmd_list, log_name):
    log_path = os.path.join(LOGS, f"{log_name}.log")
    err_path = os.path.join(LOGS, f"{log_name}.err")
    if not KAGGLE_PRESENT:
        msg = f"[DRY-RUN] Would execute: {' '.join(shlex.quote(c) for c in cmd_list)}\n"
        open(log_path, "w").write(msg); open(err_path, "w").write("")
        return 0, msg, ""
    try:
        proc = subprocess.Popen(cmd_list, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        out, err = proc.communicate()
        open(log_path, "wb").write(out or b""); open(err_path, "wb").write(err or b"")
        return proc.returncode, (out or b"").decode(), (err or b"").decode()
    except Exception as e:
        return 99, "", str(e)

# The Kaggle CLI can list submissions for a competition:
# kaggle competitions submissions -c <comp>
rc, out, err = run_cmd(["kaggle","competitions","submissions","-c", COMPETITION], "kaggle_submissions_list")
print("Submissions rc:", rc)
print("stdout (truncated):", out[:500])
print("stderr (truncated):", err[:500])

# Save raw listing for audit
open(os.path.join(ARTIFACTS, "kaggle_submissions_raw.txt"), "w").write(out if out else "")


## Parse Kaggle CLI table & merge

In [ ]:
import re, csv, io, json, os

kaggle_rows = []
table = open(os.path.join(ARTIFACTS, "kaggle_submissions_raw.txt")).read() if os.path.exists(os.path.join(ARTIFACTS, "kaggle_submissions_raw.txt")) else ""

# The Kaggle CLI prints a table; extract rows by splitting lines and using simple heuristics.
lines = [ln for ln in table.splitlines() if ln.strip()]
# Try to detect header separator line (----)
sep_idx = None
for i, ln in enumerate(lines):
    if set(ln.strip()) <= set("-|+ "):
        sep_idx = i
        break

header = []
if sep_idx is not None and sep_idx > 0:
    header_line = re.sub(r"\s+", " ", lines[sep_idx-1]).strip()
    header = header_line.split(" ")
    data_lines = lines[sep_idx+1:]
    for ln in data_lines:
        # Normalize whitespace columns (simple heuristic)
        parts = re.sub(r"\s{2,}", " | ", ln).split(" | ")
        if len(parts) >= len(header):
            row = dict(zip(header, parts[:len(header)]))
            kaggle_rows.append(row)

print("Parsed Kaggle rows:", len(kaggle_rows))

# Merge strategy: write a history CSV with both local logs and Kaggle columns (where available)
history_csv = os.path.join(ARTIFACTS, "submissions_history.csv")
all_fields = set()
for r in kaggle_rows: all_fields.update(r.keys())
for rec in records: all_fields.update(rec.keys())
all_fields = sorted(all_fields)

with open(history_csv, "w", newline="") as f:
    w = csv.DictWriter(f, fieldnames=all_fields)
    w.writeheader()
    # Prefer Kaggle rows (live) first, then append local records (with different fields)
    for r in kaggle_rows:
        w.writerow(r)
    for rec in records:
        w.writerow(rec)

print("Wrote consolidated history:", history_csv)


## Compute best submission & annotate deltas

In [ ]:
import csv, os, json

history_csv = os.path.join(ARTIFACTS, "submissions_history.csv")
best = None
rows = []
if os.path.exists(history_csv):
    with open(history_csv, newline="") as f:
        rdr = csv.DictReader(f)
        for r in rdr:
            rows.append(r)

    # Try to identify a PublicScore-like field (Kaggle CLI prints "PublicScore" for many comps)
    score_field = None
    for cand in ["PublicScore","Score","PublicScore*","Public_Score","publicScore"]:
        if rows and cand in rows[0]:
            score_field = cand; break

    # Convert scores to float if possible and sort desc
    def to_float(x):
        try: return float(x)
        except: return None

    scored = [(r, to_float(r.get(score_field))) for r in rows] if score_field else []
    scored = [(r, s) for (r, s) in scored if s is not None]
    if scored:
        scored.sort(key=lambda t: t[1], reverse=True)
        best = {"score_field": score_field, "row": scored[0][0], "score": scored[0][1]}

best_path = os.path.join(ARTIFACTS, "best_submission.json")
json.dump(best or {"note":"no scores found"}, open(best_path, "w"), indent=2)
print("Best submission summary ->", best_path)
print(json.dumps(best or {}, indent=2))


## Trend sketch (Mermaid)

> You can paste the following into your README to visualize simple submission flow.

```mermaid
flowchart TB
  A[Local submission logs] --> B[Consolidate snapshot]
  B --> C[Pull Kaggle submissions list]
  C --> D[Merge to history CSV]
  D --> E{Best score?}
  E -- yes --> F[Write best_submission.json]
  E -- no --> G[No score available]
```


## Browse produced artifacts

In [ ]:
import os

def tree(path, prefix=""):
    items = sorted(os.listdir(path))
    lines = []
    for i, name in enumerate(items):
        full = os.path.join(path, name)
        connector = "└── " if i == len(items)-1 else "├── "
        lines.append(prefix + connector + name)
        if os.path.isdir(full):
            extension = "    " if i == len(items)-1 else "│   "
            lines.extend(tree(full, prefix + extension))
    return lines

print("ARTIFACTS TREE:", ARTIFACTS)
print("\n".join(tree(ARTIFACTS)))


## Next steps
- Keep this notebook in CI after Notebook 11 to **log submissions automatically**.
- If the competition exposes a public submissions API/CSV, swap the CLI table parser for a JSON/CSV endpoint for better reliability.
- Extend the merge to include **config hash**, **data version** (from DVC), and Git tag for precise provenance.
- Consider a tiny dashboard (static HTML) that renders `submissions_history.csv` and highlights the best score per day.


## 13_gui_dashboard_demo.ipynb

# 13 · GUI Dashboard Demo (SpectraMind V50)

Thin, **CLI‑first** viewer that launches/embeds the generated diagnostics dashboard (HTML) and provides a light local server for browsing artifacts under `outputs/`.

**What this notebook does**
1) Locates a previously generated diagnostics HTML (e.g., `outputs/diagnostics/report.html` or a run‑scoped diagnostics file).
2) (Optional) Calls the CLI to generate the dashboard if missing.
3) Displays the dashboard **in‑notebook** via an iframe.
4) (Optional) Starts a simple local HTTP server to browse `outputs/` (for local dev; disabled on Kaggle).
5) Writes a small viewer **manifest** under `outputs/notebooks/13_gui_dashboard_demo/`.

> Contract: This is a **thin viewer**. No pipeline logic here — we only call the official CLI and render the resulting HTML.


In [ ]:
import os, sys, json, shutil, subprocess, platform, socket, contextlib
from pathlib import Path
from datetime import datetime

ROOT = Path.cwd().resolve()
NB_OUT = ROOT / 'outputs' / 'notebooks' / '13_gui_dashboard_demo'
NB_OUT.mkdir(parents=True, exist_ok=True)

IS_KAGGLE = Path('/kaggle/working').exists()
CLI = shutil.which('spectramind') or (f"{sys.executable} {ROOT/'spectramind.py'}" if (ROOT/'spectramind.py').exists() else f"{sys.executable} -m spectramind")

env = {
    'python': platform.python_version(),
    'platform': platform.platform(),
    'is_kaggle': IS_KAGGLE,
    'cli': CLI
}
(NB_OUT/'env_snapshot.json').write_text(json.dumps(env, indent=2))
print(json.dumps(env, indent=2))

## Parameters
Edit if you need to point at a specific run or force dashboard generation.

In [ ]:
# Where to look for an existing diagnostics HTML
REPORT_HINTS = [
    ROOT/'outputs'/'diagnostics'/'report.html',
    ROOT/'outputs',       # scan recursively for *.html with 'diagnostic' in name
]

# If True, call the CLI to (re)generate the dashboard when not found
AUTO_GENERATE_DASHBOARD = False  # toggle on if you want the notebook to call the CLI

# If generating via CLI, write to this path (adjust per your repo):
CLI_DASHBOARD_OUT = ROOT/'outputs'/'diagnostics'/'report.html'

print('REPORT_HINTS:', [str(p) for p in REPORT_HINTS])
print('AUTO_GENERATE_DASHBOARD:', AUTO_GENERATE_DASHBOARD)
print('CLI_DASHBOARD_OUT:', CLI_DASHBOARD_OUT)

## Locate existing dashboard report
We try direct paths first, then recursively search `outputs/` for files that look like diagnostics dashboards.

In [ ]:
from typing import Optional

def newest_dashboard(hints) -> Optional[Path]:
    cands = []
    for h in hints:
        if not h.exists():
            continue
        if h.is_file() and h.suffix.lower() == '.html':
            cands.append(h)
        elif h.is_dir():
            for p in h.rglob('*.html'):
                name = p.name.lower()
                if any(tok in name for tok in ('diagnostic','dashboard','report')):
                    cands.append(p)
    if not cands:
        return None
    return sorted(cands, key=lambda p: p.stat().st_mtime)[-1]

REPORT = newest_dashboard(REPORT_HINTS)
print('Found dashboard:', REPORT)

## (Optional) Generate the dashboard via CLI
This only runs when `AUTO_GENERATE_DASHBOARD=True`. Adjust the command to match your repository (e.g., `spectramind diagnose dashboard`).

In [ ]:
if AUTO_GENERATE_DASHBOARD:
    try:
        cmd = [
            *CLI.split(), 'diagnose', 'dashboard',
            f'--out', str(CLI_DASHBOARD_OUT)
        ]
        print('Running:', ' '.join(cmd))
        subprocess.run(cmd, check=True)
        REPORT = CLI_DASHBOARD_OUT if CLI_DASHBOARD_OUT.exists() else newest_dashboard([ROOT/'outputs'])
        print('Generated report ->', REPORT)
    except Exception as e:
        print('CLI dashboard generation failed (non‑blocking):', e)
else:
    print('CLI generation disabled; using existing artifacts only.')

## In‑notebook viewer
Renders the dashboard using an iframe. On Kaggle, this is the preferred way (no external ports).

In [ ]:
from IPython.display import IFrame, display, HTML

if REPORT and REPORT.exists():
    # Use relative path when possible so the iframe can resolve asset links
    try:
        rel = REPORT.relative_to(ROOT)
    except Exception:
        rel = REPORT
    print('Displaying:', rel)
    display(IFrame(src=str(rel), width='100%', height=700))
else:
    print('No diagnostics HTML found. Enable AUTO_GENERATE_DASHBOARD or populate outputs/diagnostics/.')

## (Optional) Lightweight local server (for local dev only)
Starts a simple `http.server` to browse `outputs/`. **Not** recommended on Kaggle (no external ports). Set `START_SERVER=True` only on your workstation.

In [ ]:
START_SERVER = False  # toggle True for local dev only
SERVER_ROOT = ROOT/'outputs'

def free_port(start=8000, end=8999):
    for port in range(start, end+1):
        with contextlib.closing(socket.socket(socket.AF_INET, socket.SOCK_STREAM)) as s:
            if s.connect_ex(('127.0.0.1', port)) != 0:
                return port
    return None

if START_SERVER and not IS_KAGGLE:
    port = free_port()
    if port is None:
        print('No free port found.');
    else:
        print(f'Launching local server at http://127.0.0.1:{port}/ (root={SERVER_ROOT})')
        print('Stop the server by interrupting the cell (Kernel -> Interrupt).')
        os.chdir(SERVER_ROOT)
        try:
            from http.server import ThreadingHTTPServer, SimpleHTTPRequestHandler
        except Exception:
            from http.server import HTTPServer as ThreadingHTTPServer, SimpleHTTPRequestHandler
        handler = SimpleHTTPRequestHandler
        httpd = ThreadingHTTPServer(('127.0.0.1', port), handler)
        try:
            httpd.serve_forever()
        except KeyboardInterrupt:
            print('Shutting down server...')
            httpd.server_close()
else:
    print('Local server disabled (or running on Kaggle).')

## (Optional) Streamlit/Gradio mini‑app
For a richer demo, you can spin up Streamlit/Gradio to serve your plots. This is **off** by default to avoid extra dependencies and network binding issues in shared environments.

Uncomment and adapt the code below if you need it locally.

In [ ]:
USE_STREAMLIT = False
if USE_STREAMLIT and not IS_KAGGLE:
    try:
        import streamlit as st  # ensure installed in your env
    except Exception:
        print('Install streamlit first (pip install streamlit)');
        USE_STREAMLIT = False

if USE_STREAMLIT:
    # Example: write a simple app file and run it
    app_py = NB_OUT/'dashboard_app.py'
    app_py.write_text(
        """
import streamlit as st
from pathlib import Path
st.set_page_config(layout='wide')
st.title('SpectraMind V50 — Diagnostics Dashboard')
report = Path('outputs/diagnostics/report.html')
if report.exists():
    st.components.v1.html(report.read_text(encoding='utf-8'), height=900, scrolling=True)
else:
    st.warning('No diagnostics HTML found under outputs/diagnostics/.')
        """.strip()
    )
    cmd = f"streamlit run {app_py} --server.headless=true"
    print('Launching:', cmd)
    subprocess.run(cmd, shell=True, check=False)
else:
    print('Streamlit mini‑app disabled (or not supported in this environment).')

## Manifest & (optional) DVC add
We save a simple JSON manifest for this viewer session, and optionally `dvc add` the notebook outputs for full reproducibility.

In [ ]:
manifest = {
    'timestamp_utc': datetime.utcnow().isoformat(timespec='seconds')+'Z',
    'report_path': str(REPORT) if REPORT else None,
    'cli': CLI,
    'is_kaggle': IS_KAGGLE
}
(NB_OUT/'viewer_manifest.json').write_text(json.dumps(manifest, indent=2))
print('Wrote:', NB_OUT/'viewer_manifest.json')

if shutil.which('dvc'):
    try:
        subprocess.run(['dvc','add', str(NB_OUT)], check=False)
        subprocess.run(['git','add', f'{NB_OUT}.dvc', '.gitignore'], check=False)
        subprocess.run(['dvc','status'], check=False)
        print('DVC add done (non‑blocking).')
    except Exception as e:
        print('DVC step failed (non‑blocking):', e)
else:
    print('DVC not found; skipping.')

## 14_radiation_and_noise_modeling.ipynb

# 14 · Radiation & Noise Modeling (SpectraMind V50)

Educational, **mission‑grade** notebook to illustrate how radiation environments and detector/system noise influence transit spectra and downstream diagnostics. This notebook is **pipeline‑safe**: it *reads* existing artifacts (calibrated spectra or predictions) and writes educational diagnostics under `outputs/notebooks/14_radiation_noise_modeling/`.

### Objectives
1. Summarize noise sources relevant to spaceborne spectroscopy (shot noise, read noise, dark current, cosmic rays / radiation hits, background).
2. Load one or more spectra from `outputs/` and inject controllable noise models (Poisson, Gaussian, 1/f) and **cosmic‑ray transients**.
3. Visualize impact on FFT/autocorr structure, per‑bin variance, and symbolic bands.
4. Export a compact diagnostics bundle (JSON + CSV + PNGs) for teaching and QA.

> Contract: **Thin orchestration** over CLI/outputs; no ad‑hoc calibration or model training here. For production calibration and prediction, use the dedicated notebooks and CLI.

In [ ]:
import os, sys, json, shutil, subprocess, platform, textwrap
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
try:
    import seaborn as sns
    sns.set_context('notebook'); sns.set_style('whitegrid')
except Exception:
    pass

ROOT = Path.cwd().resolve()
NB_OUT = ROOT / 'outputs' / 'notebooks' / '14_radiation_noise_modeling'
NB_OUT.mkdir(parents=True, exist_ok=True)

ENV = {
    'python': platform.python_version(),
    'platform': platform.platform(),
    'time': datetime.utcnow().isoformat()+'Z',
}
(NB_OUT/'env_snapshot.json').write_text(json.dumps(ENV, indent=2))
print('ROOT:', ROOT) ; print('NB_OUT:', NB_OUT)

## 0) Background: radiation & detector/system noise (recap)
**Noise categories (simplified):**
- **Shot noise (photon counting)**: Poisson with variance $\sigma^2 \approx N$ photons. Dominant at high flux, fundamental.
- **Read noise**: electronics/ADC noise per read; Gaussian with fixed variance per exposure/read.
- **Dark current**: thermally generated electrons; behaves like additional Poisson process with rate depending on temperature.
- **Background (zodiacal/thermal)**: adds counts and variance; often Poisson‑like per pixel/extraction window.
- **1/f (pink) noise)**: low‑frequency drift; can imprint long‑scale structure in spectra/time series.
- **Cosmic rays / radiation hits**: transient, often impulsive events (spikes/glitches) that must be detected/flagged.

We’ll illustrate how each affects a clean spectrum (or an existing prediction) by perturbing it with controlled levels and visualizing FFT/autocorr & per‑bin variance.

## 1) Load a base spectrum from outputs/
We try `outputs/` for a predictions CSV or NPY and reduce to a single `(wavelength_index, mu)` spectrum for demonstration.

If none are found, we synthesize a plausible spectrum with two absorption features for educational use.

In [ ]:
def find_candidates():
    roots = [ROOT/'outputs']
    pats = ['**/predictions.csv','**/mu.csv','**/spectra.npy','**/mu.npy']
    cands = []
    for r in roots:
        if not r.exists():
            continue
        for pat in pats:
            cands += list(r.glob(pat))
    return sorted(set(cands), key=lambda p: p.stat().st_mtime) if cands else []

def load_one_spectrum(path: Path) -> pd.DataFrame:
    if path.suffix=='.npy':
        arr = np.load(path)
        if arr.ndim==1: arr = arr[None,:]
        mu = arr[0]
        return pd.DataFrame({'wavelength_index': np.arange(len(mu)), 'mu': mu})
    df = pd.read_csv(path)
    cols = {str(c).lower(): c for c in df.columns}
    if {'planet_id','wavelength_index','mu'}.issubset(cols):
        one = df[df[cols['planet_id']]==df[cols['planet_id']].iloc[0]].copy()
        one = one.rename(columns={cols['wavelength_index']:'wavelength_index', cols['mu']:'mu'})
        return one[['wavelength_index','mu']].sort_values('wavelength_index').reset_index(drop=True)
    mu_cols = [c for c in df.columns if str(c).startswith('mu_')]
    if mu_cols:
        mu = df[mu_cols].iloc[0].to_numpy(float)
        return pd.DataFrame({'wavelength_index': np.arange(len(mu)), 'mu': mu})
    raise ValueError(f'Unsupported schema for {path}')

CANDS = find_candidates()
if not CANDS:
    # synthesize a smooth demo spectrum
    x = np.linspace(0, 1, 283)
    mu = 0.01 + 0.002*np.exp(-0.5*((x-0.35)/0.08)**2) + 0.0015*np.exp(-0.5*((x-0.75)/0.05)**2)
    base = pd.DataFrame({'wavelength_index': np.arange(283), 'mu': mu})
    base_source = 'synthetic'
    print('No outputs found; using synthetic spectrum.')
else:
    base = load_one_spectrum(CANDS[-1])
    base_source = CANDS[-1].relative_to(ROOT).as_posix()
    print('Loaded from:', base_source)

base.head()

In [ ]:
plt.figure(figsize=(10,3))
plt.plot(base['wavelength_index'], base['mu'], lw=1.5)
plt.title('Base spectrum (μ)')
plt.xlabel('wavelength index'); plt.ylabel('μ (arb)')
plt.tight_layout(); plt.savefig(NB_OUT/'base_spectrum.png', dpi=150); plt.close()
print('Saved base_spectrum.png')

## 2) Noise models
We implement simple, composable perturbations:
- **Poisson shot noise**: `y ~ Poisson(λ=S·μ) / S` with scaling `S` to get counts domain.
- **Gaussian read noise**: `N(0, σ_read)`.
- **1/f noise**: generated via colored‑noise frequency shaping.
- **Cosmic ray hits**: sparse spikes at random indices; optionally spread with small kernels.

All random draws are controlled by a fixed seed for reproducibility. Adjust parameters as needed for teaching.

In [ ]:
rng = np.random.default_rng(1234)

def add_shot_noise(mu, scale_counts=5e5):
    lam = np.clip(scale_counts*np.maximum(mu, 0), 0, None)
    y_counts = rng.poisson(lam)
    return y_counts/scale_counts

def add_read_noise(mu, sigma_read=2e-4):
    return mu + rng.normal(0.0, sigma_read, size=mu.shape)

def add_1f_noise(mu, alpha=1.0, amp=2e-4):
    n = len(mu)
    white = rng.normal(0,1,n)
    f = np.fft.rfftfreq(n)
    spec = np.fft.rfft(white)
    with np.errstate(divide='ignore', invalid='ignore'):
        shaping = 1.0/np.maximum(f, 1e-6)**alpha
    shaped = spec*shaping
    x = np.fft.irfft(shaped, n)
    x = amp*x/np.std(x)
    return mu + x

def add_cosmic_rays(mu, n_hits=3, spike_amp=0.01, kernel=[1.0, 0.5]):
    y = mu.copy()
    W = len(mu)
    if n_hits <= 0:
        return y, np.array([], dtype=int)
    hits = rng.choice(W, size=min(n_hits,W), replace=False)
    for h in hits:
        for k,a in enumerate(kernel):
            idx = h+k
            if idx < W:
                y[idx] += a*spike_amp
    return y, hits

mu = base['mu'].to_numpy(float)
noisy_shot   = add_shot_noise(mu, scale_counts=3e5)
noisy_read   = add_read_noise(mu, sigma_read=2e-4)
noisy_1f     = add_1f_noise(mu, alpha=1.0, amp=2e-4)
noisy_cr, H  = add_cosmic_rays(mu, n_hits=4, spike_amp=0.01)
H

In [ ]:
x = base['wavelength_index']
fig, ax = plt.subplots(2,2, figsize=(12,6), sharex=True)
ax = ax.ravel()
ax[0].plot(x, mu, lw=1.2, label='base')
ax[0].plot(x, noisy_shot, lw=0.8, label='shot')
ax[0].set_title('Shot noise') ; ax[0].legend()

ax[1].plot(x, mu, lw=1.2, label='base')
ax[1].plot(x, noisy_read, lw=0.8, label='read')
ax[1].set_title('Read noise') ; ax[1].legend()

ax[2].plot(x, mu, lw=1.2, label='base')
ax[2].plot(x, noisy_1f, lw=0.8, label='1/f')
ax[2].set_title('1/f noise') ; ax[2].legend()

ax[3].plot(x, mu, lw=1.2, label='base')
ax[3].plot(x, noisy_cr, lw=0.8, label='cosmic rays')
if H.size:
    ax[3].scatter(x.iloc[H], noisy_cr[H], s=20, zorder=3)
ax[3].set_title('Radiation hits (spikes)') ; ax[3].legend()

for a in ax: a.set_ylabel('μ (arb)')
ax[2].set_xlabel('wavelength index'); ax[3].set_xlabel('wavelength index')
fig.tight_layout(); fig.savefig(NB_OUT/'noise_panels.png', dpi=150); plt.close(fig)
print('Saved noise_panels.png')

## 3) FFT & autocorrelation impact
We reuse the simple FFT/AC routines used elsewhere to illustrate how each noise class alters spectral frequency content and lag structure.

In [ ]:
def fft_power_onesided(y):
    y = np.asarray(y, float)
    y = y - np.nanmean(y)
    fy = np.fft.rfft(y)
    power = np.abs(fy)**2
    freqs = np.fft.rfftfreq(len(y))
    return freqs, power

def autocorr_norm(y):
    y = np.asarray(y, float)
    y = y - np.nanmean(y)
    r = np.correlate(y, y, mode='full')
    r = r[r.size//2:]
    if r[0] != 0:
        r = r / r[0]
    lags = np.arange(r.size)
    return lags, r

series = {
    'base': mu,
    'shot': noisy_shot,
    'read': noisy_read,
    '1f':   noisy_1f,
    'cr':   noisy_cr
}

fig, ax = plt.subplots(2,2, figsize=(12,6))
ax = ax.ravel()
for i,(name, y) in enumerate(series.items()):
    f,p = fft_power_onesided(y)
    ax[i//2].semilogy(f[1:], p[1:], lw=1, label=name)
ax[0].set_title('FFT power (A)') ; ax[1].set_title('FFT power (B)')
ax[0].legend(); ax[1].legend()
for a in ax[:2]: a.set_xlabel('freq'); a.set_ylabel('power')

for i,(name, y) in enumerate(series.items()):
    l,r = autocorr_norm(y)
    ax[2 + (i%2)].plot(l, r, lw=1, label=name)
ax[2].set_title('Autocorr (A)') ; ax[3].set_title('Autocorr (B)')
ax[2].legend(); ax[3].legend()
for a in ax[2:]: a.set_xlabel('lag'); a.set_ylabel('norm acorr')
fig.tight_layout(); fig.savefig(NB_OUT/'fft_autocorr_noise_compare.png', dpi=150); plt.close(fig)
print('Saved fft_autocorr_noise_compare.png')

## 4) Per‑bin variance & symbolic band overlays
We compute per‑bin variance across a small ensemble of perturbed spectra and (optionally) overlay symbolic bands (e.g., water) to show if noise masks lines of interest.

In [ ]:
def ensemble_variance(mu, K=64, cfg=None):
    cfg = cfg or {}
    ens = []
    for k in range(K):
        y = mu.copy()
        if cfg.get('shot'):   y = add_shot_noise(y, scale_counts=cfg.get('shot_scale',3e5))
        if cfg.get('read'):   y = add_read_noise(y, sigma_read=cfg.get('read_sigma',2e-4))
        if cfg.get('one_over_f'): y = add_1f_noise(y, alpha=cfg.get('alpha',1.0), amp=cfg.get('amp',2e-4))
        if cfg.get('cosmic_hits'):
            y,_ = add_cosmic_rays(y, n_hits=cfg.get('cr_n',3), spike_amp=cfg.get('cr_amp',0.01))
        ens.append(y)
    ens = np.stack(ens, axis=0)
    return ens.var(axis=0)

cfg = {'shot':True, 'read':True, 'one_over_f':True, 'cosmic_hits':True}
var_bins = ensemble_variance(mu, K=64, cfg=cfg)
plt.figure(figsize=(10,3))
plt.plot(base['wavelength_index'], var_bins, lw=1)
plt.title('Per‑bin variance under composite noise model')
plt.xlabel('wavelength index'); plt.ylabel('variance')
plt.tight_layout(); plt.savefig(NB_OUT/'perbin_variance.png', dpi=150); plt.close()
print('Saved perbin_variance.png')

## 5) Bundle export
We export a compact JSON + CSV for dashboards and lessons learned.

In [ ]:
bundle = {
  'source': base_source,
  'noise_panels': 'noise_panels.png',
  'fft_autocorr_compare': 'fft_autocorr_noise_compare.png',
  'perbin_variance': 'perbin_variance.png',
  'notes': 'Educational demo; parameters are illustrative and not instrument‑calibrated.'
}
(NB_OUT/'radiation_noise_bundle.json').write_text(json.dumps(bundle, indent=2))
pd.DataFrame({'wavelength_index': base['wavelength_index'], 'mu_base': mu, 'var_noise': var_bins}).to_csv(NB_OUT/'radiation_noise_detail.csv', index=False)
print('Wrote bundle & detail CSV to', NB_OUT)

---
### Notes & Further Reading
- **Shot/read/dark/background** processes and their statistics are standard detector topics; see your instrument handbook for exact models and units.
- **Cosmic ray** rates depend on orbit and shielding; spikes must be detected/flagged to avoid biasing spectra and FFT/autocorr diagnostics.
- For production pipeline, rely on the **calibration kill chain** and CLI diagnostics rather than these educational injectors.

## 15_gravitational_lensing_demo.ipynb

# 15 · Gravitational Lensing Demo (SpectraMind V50)

Educational, **mission‑grade** demo that shows how a simple microlensing event can bias *observed* transit data products when sampling/weighting effects are present. This notebook is **pipeline‑safe**:

- It **does not** implement pipeline logic; it *reads* artifacts produced by the CLI (if present) or synthesizes a clean spectrum.
- It simulates a **point‑lens microlensing** light curve during transit and demonstrates how temporal magnification + wavelength‑dependent weights can create apparent spectral biases.
- Artifacts are written under `outputs/notebooks/15_gravitational_lensing/` for DVC tracking.

**What you'll do**
1. Load a base transmission spectrum (from `outputs/` or synthesize).
2. Simulate microlensing magnification `A(t)` with typical point‑lens formula.
3. Combine `A(t)` with a simple exposure model and wavelength‑dependent weights to build an **effective** spectrum.
4. Compare original vs lensed spectra; visualize Einstein‑curve, residuals, and band overlays.
5. Export a compact diagnostics bundle (JSON + CSV + PNGs).

In [ ]:
import os, sys, json, platform
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
try:
    import seaborn as sns
    sns.set_context('notebook'); sns.set_style('whitegrid')
except Exception:
    pass

ROOT = Path.cwd().resolve()
NB_OUT = ROOT / 'outputs' / 'notebooks' / '15_gravitational_lensing'
NB_OUT.mkdir(parents=True, exist_ok=True)

ENV = {'python': platform.python_version(), 'platform': platform.platform(), 'time': __import__('datetime').datetime.utcnow().isoformat()+'Z'}
(NB_OUT/'env_snapshot.json').write_text(json.dumps(ENV, indent=2))
print('ROOT:', ROOT) ; print('NB_OUT:', NB_OUT)

## 0) Background: point‑lens microlensing
For a point lens, the (achromatic) magnification for a projected separation `u = θ/θ_E` is:

$$A(u) = \frac{u^2 + 2}{u\,\sqrt{u^2 + 4}}\,.$$ 

With constant proper motion, `u(t) = \sqrt{u_0^2 + ((t - t_0)/t_E)^2}`. In reality lensing is achromatic, but **effective** spectral biases can arise if magnification varies during a transit while the instrument/pipeline applies time‑/wavelength‑dependent weights (exposure timing, throughput, flagging). We illustrate this effect using a simplified toy model.

## 1) Load base spectrum
We attempt to load a predictions table from `outputs/` and select one spectrum; if not found, we synthesize a smooth spectrum with two absorption features.

In [ ]:
def find_candidates():
    roots = [ROOT/'outputs']
    pats = ['**/predictions.csv','**/mu.csv','**/spectra.npy','**/mu.npy']
    cands = []
    for r in roots:
        if not r.exists():
            continue
        for pat in pats:
            cands += list(r.glob(pat))
    return sorted(set(cands), key=lambda p: p.stat().st_mtime) if cands else []

def load_one_spectrum(path: Path) -> pd.DataFrame:
    if path.suffix=='.npy':
        arr = np.load(path)
        if arr.ndim==1: arr = arr[None,:]
        mu = arr[0]
        return pd.DataFrame({'wavelength_index': np.arange(len(mu)), 'mu': mu})
    df = pd.read_csv(path)
    cols = {str(c).lower(): c for c in df.columns}
    if {'planet_id','wavelength_index','mu'}.issubset(cols):
        one = df[df[cols['planet_id']]==df[cols['planet_id']].iloc[0]].copy()
        one = one.rename(columns={cols['wavelength_index']:'wavelength_index', cols['mu']:'mu'})
        return one[['wavelength_index','mu']].sort_values('wavelength_index').reset_index(drop=True)
    mu_cols = [c for c in df.columns if str(c).startswith('mu_')]
    if mu_cols:
        mu = df[mu_cols].iloc[0].to_numpy(float)
        return pd.DataFrame({'wavelength_index': np.arange(len(mu)), 'mu': mu})
    raise ValueError(f'Unsupported schema for {path}')

CANDS = find_candidates()
if not CANDS:
    # synthesize a clean spectrum (283 bins) with two features
    x = np.linspace(0, 1, 283)
    mu_clean = 0.01 + 0.002*np.exp(-0.5*((x-0.30)/0.06)**2) + 0.0015*np.exp(-0.5*((x-0.70)/0.05)**2)
    base = pd.DataFrame({'wavelength_index': np.arange(283), 'mu': mu_clean})
    base_source = 'synthetic'
    print('No outputs found; using synthetic spectrum.')
else:
    base = load_one_spectrum(CANDS[-1])
    base_source = CANDS[-1].relative_to(ROOT).as_posix()
    print('Loaded from:', base_source)
base.head()

In [ ]:
plt.figure(figsize=(10,3))
plt.plot(base['wavelength_index'], base['mu'], lw=1.5)
plt.title('Base transmission spectrum (μ)')
plt.xlabel('wavelength index'); plt.ylabel('μ (arb)')
plt.tight_layout(); plt.savefig(NB_OUT/'base_spectrum.png', dpi=150); plt.close()
print('Saved base_spectrum.png')

## 2) Microlensing model in time
We simulate a simple transit time series with uniform exposure spacing and a point‑lens magnification that varies across the observing window.

**Parameters** (tweak to taste):
- `u0` : minimum impact parameter in Einstein‑radius units (smaller → stronger peak).
- `tE` : Einstein timescale half‑width (controls event duration).
- `t0` : time of closest approach (center of peak).

In [ ]:
def A_point_lens(u):
    u = np.asarray(u, float)
    return (u*u + 2) / (u * np.sqrt(u*u + 4))

def u_of_t(t, t0=0.0, u0=0.2, tE=0.3):
    return np.sqrt(u0*u0 + ((t - t0)/tE)**2)

# Exposure grid (normalized time)
N_EXP = 200
t = np.linspace(-0.8, 0.8, N_EXP)
A = A_point_lens(u_of_t(t, t0=0.0, u0=0.20, tE=0.25))

plt.figure(figsize=(8,3))
plt.plot(t, A, lw=1.6)
plt.title('Microlensing magnification A(t)')
plt.xlabel('time [arb]'); plt.ylabel('A')
plt.tight_layout(); plt.savefig(NB_OUT/'magnification_curve.png', dpi=150); plt.close()
print('Saved magnification_curve.png')

## 3) From time to effective spectrum
In an ideal ratio measurement, pure lensing would cancel out in the transit depth. In practice, **time‑varying magnification** during the observation coupled with **wavelength‑dependent exposure/weighting** (due to throughput, flagging, or rolling shutter timing) can produce wavelength‑dependent biases.

We model this by defining per‑wavelength exposure weights `W(λ)` (normalized) and computing a weighted average across time windows that intersect the lensing peak. This toy model demonstrates a possible **effective** bias.

In [ ]:
wl = base['wavelength_index'].to_numpy()
mu = base['mu'].to_numpy(float)
W = wl / wl.max()  # simple monotonic weight vs wavelength (proxy for throughput)
W = (W - W.min())/(W.ptp() + 1e-12)
W = 0.5 + 0.5*W  # scale to [0.5,1.0]

# Effective lensed spectrum: weight time samples by A(t) and wavelength weights W(λ)
# Simplified: pretend each λ samples a slightly shifted time slice (e.g., readout order)
phase_per_wl = np.linspace(-0.15, 0.15, len(wl))  # mock readout time offset per wavelength
A_per_wl = np.interp(phase_per_wl, t, A)

mu_lensed = mu * (1.0 + (A_per_wl - 1.0)* (W))  # multiplicative bias modulated by weights
residual = mu_lensed - mu

fig, ax = plt.subplots(2,1, figsize=(10,6), sharex=True)
ax[0].plot(wl, mu, lw=1.4, label='base μ')
ax[0].plot(wl, mu_lensed, lw=1.2, label='effective μ (with lensing+weights)')
ax[0].set_ylabel('μ (arb)'); ax[0].legend()
ax[1].plot(wl, residual, lw=1.2, label='residual (lensed − base)')
ax[1].axhline(0, color='k', lw=0.7)
ax[1].set_xlabel('wavelength index'); ax[1].set_ylabel('Δμ (arb)'); ax[1].legend()
fig.suptitle('Effective spectrum under microlensing + wavelength weights')
fig.tight_layout(); fig.savefig(NB_OUT/'effective_spectrum_lensing.png', dpi=150); plt.close(fig)
print('Saved effective_spectrum_lensing.png')

### Symbolic band overlays (educational)
We overlay two nominal water‑band index ranges to show whether apparent residuals cluster in key regions. Replace with instrument‑accurate bands for your grid.

In [ ]:
SYM_BANDS = {
    'H2O_1': (120, 150),
    'H2O_2': (180, 220)
}

plt.figure(figsize=(10,3))
plt.plot(wl, residual, lw=1.2)
for name,(a,b) in SYM_BANDS.items():
    a,b = max(0,a), min(len(wl),b)
    if b>a:
        plt.axvspan(wl[a], wl[b-1], alpha=0.18)
        plt.text((wl[a]+wl[b-1])/2, residual.max()*0.9 if residual.max()!=0 else 0.0, name, ha='center', va='top', fontsize=8, alpha=0.8)
plt.axhline(0, lw=0.7)
plt.title('Residuals with symbolic band overlays')
plt.xlabel('wavelength index'); plt.ylabel('Δμ (arb)')
plt.tight_layout(); plt.savefig(NB_OUT/'residuals_with_bands.png', dpi=150); plt.close()
print('Saved residuals_with_bands.png')

## 4) Einstein‑ring radius sketch (quick look)
We plot the classic point‑lens image separation scale as context (qualitative sketch; not used directly in spectra).

In [ ]:
theta = np.linspace(0, 2*np.pi, 400)
x = np.cos(theta)
y = np.sin(theta)
plt.figure(figsize=(4,4))
plt.plot(x, y, lw=1.5)
plt.gca().set_aspect('equal', adjustable='box')
plt.title('Einstein ring (unit radius sketch)')
plt.axis('off')
plt.tight_layout(); plt.savefig(NB_OUT/'einstein_ring_sketch.png', dpi=150); plt.close()
print('Saved einstein_ring_sketch.png')

## 5) Export bundle
We save a compact JSON bundle and CSV for dashboards/teaching.

In [ ]:
bundle = {
  'base_source': base_source,
  'n_exp': int(len(np.linspace(-0.8,0.8,200))),
  'figures': [
      'base_spectrum.png',
      'magnification_curve.png',
      'effective_spectrum_lensing.png',
      'residuals_with_bands.png',
      'einstein_ring_sketch.png'
  ],
  'notes': 'Toy model: achromatic microlensing + wavelength/time weighting may yield effective spectral bias.'
}
(NB_OUT/'lensing_demo_bundle.json').write_text(json.dumps(bundle, indent=2))
import pandas as pd
pd.DataFrame({'wavelength_index': wl, 'mu_base': mu, 'mu_lensed': mu_lensed, 'residual': residual}).to_csv(NB_OUT/'lensing_demo_detail.csv', index=False)
print('Wrote bundle & detail CSV to', NB_OUT)

## 6) (Optional) DVC add
Register outputs for full reproducibility if your project uses DVC.

In [ ]:
import shutil, subprocess
if shutil.which('dvc'):
    try:
        subprocess.run(['dvc','add', str(NB_OUT)], check=False)
        subprocess.run(['git','add', f'{NB_OUT}.dvc', '.gitignore'], check=False)
        subprocess.run(['dvc','status'], check=False)
        print('DVC add done (non‑blocking).')
    except Exception as e:
        print('DVC step failed (non‑blocking):', e)
else:
    print('DVC not found; skipping.')

---
### Notes & caveats
- **Achromatic** lensing by itself does *not* change intrinsic spectral features; apparent biases here arise from simplified time/weight coupling used for demonstration.
- For realistic use, replace the weight model `W(λ)` and timing offsets with instrument‑specific readout/throughput and sampling.
- Keep production spectra in the CLI calibration/prediction pipeline; use this notebook for physics intuition and QA visualization only.